In [1]:
# initialization and random seed set

import os
import scipy
import torch
import random
import sklearn
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from CBIG_model_pytorch import dnn_4l, dnn_5l
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Step 1. Load Data

In [2]:
subejct_df = pd.read_csv('for_reproduce/MMP_HCP_753_Age_list.csv', index_col = 0)
#unrestricted_sophiafrangou_7_11_2021_22_0_0
phenotype = pd.read_csv('for_reproduce/unrestricted_sophiafrangou_7_11_2021_22_0_0.csv', index_col = 0)
phenotype.head()

,Release,Acquisition,Gender,Age,3T_Full_MR_Compl,T1_Count,T2_Count,3T_RS-fMRI_Count,3T_RS-fMRI_PctCompl,3T_Full_Task_fMRI,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
Subject,,,,,,,,,,,,,,,,,,,,,
100004,S900,Q06,M,22-25,False,0,0,0,0.0,False,...,5.2,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80
100206,S900,Q11,M,26-30,True,1,1,4,100.0,True,...,6.0,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
100307,Q1,Q01,F,26-30,True,1,1,4,100.0,True,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
100408,Q3,Q03,M,31-35,True,1,1,4,100.0,True,...,2.0,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
100610,S900,Q08,M,26-30,True,2,1,4,100.0,True,...,2.0,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88


In [3]:
pheno_list = ['PicSeq_Unadj', 'CardSort_Unadj', 'Flanker_Unadj',
 'PMAT24_A_CR', 'ReadEng_Unadj', 'PicVocab_Unadj', 'ProcSpeed_Unadj',
 'DDisc_AUC_40K', 'VSPLOT_TC', 'SCPT_SEN', 'SCPT_SPEC',
 'IWRD_TOT', 'ListSort_Unadj', 'MMSE_Score', 'PSQI_Score', 'Endurance_Unadj',
 'GaitSpeed_Comp', 'Dexterity_Unadj', 'Strength_Unadj', 'Odor_Unadj', 'PainInterf_Tscore',
 'Taste_Unadj', 'Mars_Final', 'Emotion_Task_Face_Acc',
 'Language_Task_Math_Avg_Difficulty_Level', 'Language_Task_Story_Avg_Difficulty_Level',
 'Relational_Task_Acc', 'Social_Task_Perc_Random',
 'Social_Task_Perc_TOM', 'WM_Task_Acc', 'NEOFAC_A', 'NEOFAC_O',
 'NEOFAC_C', 'NEOFAC_N', 'NEOFAC_E', 'ER40_CR', 'ER40ANG', 'ER40FEAR', 'ER40HAP', 'ER40NOE',
 'ER40SAD', 'AngAffect_Unadj', 'AngHostil_Unadj', 'AngAggr_Unadj', 'FearAffect_Unadj',
 'FearSomat_Unadj', 'Sadness_Unadj', 'LifeSatisf_Unadj',
 'MeanPurp_Unadj', 'PosAffect_Unadj', 'Friendship_Unadj', 'Loneliness_Unadj',
 'PercHostil_Unadj', 'PercReject_Unadj', 'EmotSupp_Unadj', 'InstruSupp_Unadj',
 'PercStress_Unadj', 'SelfEff_Unadj']

pheno_58 = phenotype.loc[:, pheno_list]
pheno_58

,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,PMAT24_A_CR,ReadEng_Unadj,PicVocab_Unadj,ProcSpeed_Unadj,DDisc_AUC_40K,VSPLOT_TC,SCPT_SEN,...,MeanPurp_Unadj,PosAffect_Unadj,Friendship_Unadj,Loneliness_Unadj,PercHostil_Unadj,PercReject_Unadj,EmotSupp_Unadj,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj
Subject,,,,,,,,,,,,,,,,,,,,,
100004,118.70,103.17,121.97,19.0,109.9589,111.1118,92.72,0.067448,9.0,0.8833,...,40.1,36.2,27.6,73.0,60.0,70.4,29.3,34.8,55.4,39.6
100206,125.07,119.14,130.42,20.0,113.5460,119.8914,138.72,0.050000,12.0,0.9667,...,50.0,48.4,45.7,63.8,63.8,64.8,43.9,46.0,57.8,44.0
100307,110.61,123.75,112.56,17.0,111.0700,107.2000,122.65,0.311459,9.0,0.9833,...,42.4,46.2,62.0,53.7,52.7,54.1,45.0,52.3,57.9,37.5
100408,125.71,111.14,121.18,7.0,131.8100,134.2400,107.08,0.421354,17.0,0.8667,...,51.9,48.4,52.9,51.9,52.2,50.1,50.1,51.1,46.8,56.4
100610,109.04,129.84,126.53,23.0,141.3166,140.8151,111.11,0.868750,18.0,1.0000,...,46.8,55.1,44.9,53.5,38.5,54.1,56.8,48.4,37.8,53.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992774,89.89,126.37,124.64,14.0,108.6900,106.0900,114.31,0.019531,9.0,0.9333,...,50.0,48.4,47.5,49.6,52.5,50.6,51.8,46.7,46.7,54.7
993675,108.89,120.35,116.43,21.0,129.4080,135.6653,113.40,0.938281,21.0,0.9833,...,59.8,46.2,50.7,53.7,53.9,54.1,44.8,45.9,49.8,45.9
994273,87.10,112.17,123.22,20.0,141.3166,122.3772,83.25,0.529427,13.0,0.9500,...,38.8,46.5,55.2,53.7,62.5,40.7,51.8,62.9,60.4,48.0


In [4]:
subject_list = subejct_df.index.tolist()
pheno_58 = pheno_58.loc[subject_list, :]
pheno_58.shape

(753, 58)

In [5]:
age = subejct_df[['Age']]

In [6]:
pheno_with_age = pd.concat([pheno_58, age], axis = 1)
pheno_with_age

,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,PMAT24_A_CR,ReadEng_Unadj,PicVocab_Unadj,ProcSpeed_Unadj,DDisc_AUC_40K,VSPLOT_TC,SCPT_SEN,...,PosAffect_Unadj,Friendship_Unadj,Loneliness_Unadj,PercHostil_Unadj,PercReject_Unadj,EmotSupp_Unadj,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj,Age
Subject,,,,,,,,,,,,,,,,,,,,,
100206,125.07,119.14,130.42,20.0,113.5460,119.89140,138.72,0.050000,12.0,0.9667,...,48.4,45.7,63.8,63.8,64.8,43.9,46.0,57.8,44.0,27
100307,110.61,123.75,112.56,17.0,111.0700,107.20000,122.65,0.311459,9.0,0.9833,...,46.2,62.0,53.7,52.7,54.1,45.0,52.3,57.9,37.5,27
100408,125.71,111.14,121.18,7.0,131.8100,134.24000,107.08,0.421354,17.0,0.8667,...,48.4,52.9,51.9,52.2,50.1,50.1,51.1,46.8,56.4,33
101006,84.68,105.19,101.85,11.0,113.5374,95.42348,90.59,0.783073,20.0,0.9833,...,42.0,57.1,50.3,46.1,46.5,62.5,43.3,48.8,38.8,35
101107,105.60,119.76,107.04,14.0,119.2434,119.84590,112.27,0.584375,16.0,0.9167,...,59.2,66.5,52.0,63.8,43.7,62.5,32.6,51.9,68.4,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992673,96.55,94.96,94.87,21.0,117.9919,118.44630,104.06,0.791667,11.0,0.8833,...,50.2,49.8,50.3,47.8,40.7,62.5,46.7,41.0,49.3,33
992774,89.89,126.37,124.64,14.0,108.6900,106.09000,114.31,0.019531,9.0,0.9333,...,48.4,47.5,49.6,52.5,50.6,51.8,46.7,46.7,54.7,35
993675,108.89,120.35,116.43,21.0,129.4080,135.66530,113.40,0.938281,21.0,0.9833,...,46.2,50.7,53.7,53.9,54.1,44.8,45.9,49.8,45.9,29


In [7]:
cort_thick = pd.read_csv('./pca_thres8_csv/cort_thick_thres8.csv', encoding = 'cp949', index_col=0)
cort_area = pd.read_csv('./pca_thres8_csv/cort_area_thres8.csv', encoding = 'cp949', index_col=0)
cort_vol = pd.read_csv('./pca_thres8_csv/cort_vol_thres8.csv', encoding = 'cp949', index_col=0)


tbss_ad = pd.read_csv('./pca_thres8_csv/tbss_ad_thres8.csv')
tbss_fa = pd.read_csv('./pca_thres8_csv/tbss_fa_thres8.csv', encoding = 'cp949', index_col=0)
tbss_icvf = pd.read_csv('./pca_thres8_csv/tbss_icvf_thres8.csv', encoding = 'cp949', index_col=0)
tbss_isovf = pd.read_csv('./pca_thres8_csv/tbss_isovf_thres8.csv', encoding = 'cp949', index_col=0)
tbss_md = pd.read_csv('./pca_thres8_csv/tbss_md_thres8.csv', encoding = 'cp949', index_col=0)
tbss_od = pd.read_csv('./pca_thres8_csv/tbss_od_thres8.csv', encoding = 'cp949', index_col=0)
tbss_rd = pd.read_csv('./pca_thres8_csv/tbss_rd_thres8.csv', encoding = 'cp949', index_col=0)


f_gamb = pd.read_csv('./pca_thres8_csv/f_gamb_thres8.csv', encoding = 'cp949', index_col=0)
f_lang = pd.read_csv('./pca_thres8_csv/f_lang_thres8.csv', encoding = 'cp949', index_col=0)
f_motor = pd.read_csv('./pca_thres8_csv/f_motor_thres8.csv', encoding = 'cp949', index_col=0)
f_rs = pd.read_csv('./pca_thres8_csv/f_rs_thres8.csv', encoding = 'cp949', index_col=0)
f_social = pd.read_csv('./pca_thres8_csv/f_social_thres8.csv', encoding = 'cp949', index_col=0)
f_wm = pd.read_csv('./pca_thres8_csv/f_wm_thres8.csv', encoding = 'cp949', index_col=0)


tractography_ad = pd.read_csv('./pca_thres8_csv/tractography_ad_thres8.csv', encoding = 'cp949', index_col=0)
tractography_fa = pd.read_csv('./pca_thres8_csv/tractography_fa_thres8.csv', encoding = 'cp949', index_col=0)
tractography_icvf = pd.read_csv('./pca_thres8_csv/tractography_icvf_thres8.csv', encoding = 'cp949', index_col=0)
tractography_isovf = pd.read_csv('./pca_thres8_csv/tractography_isovf_thres8.csv', encoding = 'cp949', index_col=0)
tractography_md = pd.read_csv('./pca_thres8_csv/tractography_md_thres8.csv', encoding = 'cp949', index_col=0)
tractography_od = pd.read_csv('./pca_thres8_csv/tractography_od_thres8.csv', encoding = 'cp949', index_col=0)
tractography_rd = pd.read_csv('./pca_thres8_csv/tractography_rd_thres8.csv', encoding = 'cp949', index_col=0)
tractography_fss = pd.read_csv('./pca_thres8_csv/tractography_fss_thres8.csv', encoding = 'cp949', index_col=0)
tractography_fssl = pd.read_csv('./pca_thres8_csv/tractography_fssl_thres8.csv', encoding = 'cp949', index_col=0)

In [8]:
pd.DataFrame(cort_thick).head()

,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,...,pca126,pca127,pca128,pca129,pca130,pca131,pca132,pca133,pca134,pca135
0,2.993363,0.750918,-4.877755,-3.991837,3.621244,0.037990,0.829940,-0.297033,0.500779,-1.026544,...,-0.475529,-1.797621,0.041214,0.348750,-0.801119,-0.541519,0.601518,0.374228,0.815612,-0.948277
1,-8.459897,9.607804,-4.791172,1.786094,1.268059,-0.741211,-1.241229,-2.684130,-0.293249,0.752931,...,1.309966,-1.422582,-1.011954,1.143709,-1.382897,-0.249345,-0.495145,0.170680,0.601043,0.266287
2,8.525777,-2.826453,0.567971,-2.275832,1.610122,-0.221138,-1.698250,1.377701,1.113820,-1.188727,...,-0.316786,-0.299607,-1.055971,0.810313,-0.110750,1.488462,-0.186271,-1.025785,0.574387,0.776836
3,5.745110,1.924774,4.759713,0.097288,-3.408099,1.348350,1.048009,-0.811042,-2.938523,-4.809545,...,0.463992,0.959571,-1.389049,1.426847,-0.144285,-1.142545,0.044784,0.619992,-0.099348,1.603124
4,-18.038552,-2.682422,-0.991662,0.183546,-1.084038,1.669913,-2.128331,-0.505386,2.254149,3.232299,...,0.713627,0.209927,0.220733,0.604531,0.726827,-1.572306,-0.559661,-0.414037,-0.070604,-0.789170


In [9]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

cuda


In [10]:
def make_meta(df, pheno_with_age, subject_list, seed):
    random.seed(seed)
    df = pd.DataFrame(df)
    df.index = subject_list
    
    random.shuffle(subject_list)
    
    df = df.reindex(subject_list)
    pheno_with_age = pheno_with_age.reindex(subject_list)
    
    meta_x_data = df.to_numpy()
    meta_y_data = pheno_with_age.to_numpy()
    
    return meta_x_data, meta_y_data

### Scaling 및 Pre-processing

In [44]:
def dataset_preprocessing(meta_x_data, meta_y_data, k_num, seed):
    
    x_train, x_test, y_train, y_test = train_test_split(meta_x_data, meta_y_data, test_size=0.2, random_state=seed)
    
    # Train meta set을 Train / Validation set split 
    train_meta_x = x_train[:int(x_train.shape[0] * 0.8)]
    val_meta_x = x_train[int(x_train.shape[0] * 0.8):]
    train_meta_y = y_train[:int(y_train.shape[0] * 0.8), :-1]
    val_meta_y = y_train[int(y_train.shape[0] * 0.8):, :-1]
    
    # Train set과 Validation set의 pheno_with_age를 StandardScaling
    train_scaler = StandardScaler()
    train_scaler.fit(train_meta_y)
    train_meta_y = train_scaler.transform(train_meta_y)
    val_meta_y = train_scaler.transform(val_meta_y)
    
    # Test meta set을 K-train과 rest_train_meta로 분리 
    k_train_meta = x_test[:k_num] # k * smri(400) 
    rest_train_meta = x_test[k_num:] # rest * smri(400)
    
    
    test_scaler = StandardScaler()
    test_scaler.fit(y_test[:k_num])
    k_shot_meta_y = test_scaler.transform(y_test[:k_num])
    test_meta_y = test_scaler.transform(y_test[k_num:])        

    # _val_ : Age를 제외한 Phenotype
    # _test_ : Age에 대한 Series 
    k_val_meta = k_shot_meta_y[:, :-1] # k * 58
    k_test_meta = k_shot_meta_y[:, -1] # k * age(1)
    rest_val_meta = test_meta_y[:, :-1] # rest * 58
    rest_test_meta = test_meta_y[:, -1] # rest * age(1)
    
    return train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta

### Scaling 및 Pre-processing in Fine-Tuning Ver.

In [12]:
def dataset_finetune_preprocessing(meta_x_data, meta_y_data, k_num, seed):
    
    x_train, x_test, y_train, y_test = train_test_split(meta_x_data, meta_y_data, test_size=0.2, random_state=seed)
    
    # Train meta set을 Train / Validation set split 
    train_meta_x = x_train[:int(x_train.shape[0] * 0.8)]
    val_meta_x = x_train[int(x_train.shape[0] * 0.8):]
    train_meta_y = y_train[:int(y_train.shape[0] * 0.8), :-1]
    val_meta_y = y_train[int(y_train.shape[0] * 0.8):, :-1]
    
    # Train set과 Validation set의 pheno_with_age를 StandardScaling
    train_scaler = StandardScaler()
    train_scaler.fit(train_meta_y)
    train_meta_y = train_scaler.transform(train_meta_y)
    val_meta_y = train_scaler.transform(val_meta_y)
    
    # Test meta set을 K-train과 rest_train_meta로 분리 
    k_train_meta = x_test[:k_num] # k * smri(400) 
    rest_train_meta = x_test[k_num:] # rest * smri(400)
    

    
    test_scaler = StandardScaler()
    test_scaler.fit(y_test[:k_num])
    k_shot_meta_y = test_scaler.transform(y_test[:k_num])
    test_meta_y = test_scaler.transform(y_test[k_num:])        

    # _val_ : Age를 제외한 Phenotype
    # _test_ : Age에 대한 Series 
    k_val_meta = k_shot_meta_y[:, :-1] # k * 58
    k_test_meta = k_shot_meta_y[:, -1] # k * age(1)
    rest_val_meta = test_meta_y[:, :-1] # rest * 58
    rest_test_meta = test_meta_y[:, -1] # rest * age(1)
    
    # k-shot을 finetuning 위해 train/validation split
    ft_train_x, ft_val_x, ft_train_y, ft_val_y = train_test_split(k_train_meta, k_shot_meta_y, test_size=0.2, random_state=seed)
    ft_age_x = ft_train_y[:, -1]
    ft_age_y = ft_val_y[:, -1]
    ft_train_y = ft_train_y[:, :-1]
    ft_val_y = ft_val_y[:, :-1]
            
    return train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta, ft_train_x, ft_val_x, ft_train_y, ft_val_y, ft_age_x, ft_age_y

### Basic KRR

In [13]:
# Age file 
age_file = pd.read_csv('for_reproduce/MMP_HCP_753_Age_list.csv', index_col = 0)
age_file = age_file[['Age']].reset_index()

def basic_krr(age_file, anat, pheno, model, k, iteration=1):
    print(f"\n\n Experiment : k={k}, iteration={iteration}")
    age_file_col_num = 1
    anat_col_num = anat.shape[1]
    pheno_col_num = 58
    
    
    age_anat = pd.concat([age_file, anat], axis=1)
    age_anat_pheno = pd.merge(age_anat, pheno_58, on='Subject', how='inner')
    
    final_corr_list = []
    
    # for seed in tqdm(range(iteration)):
    for seed in range(iteration):
        # Seed
        X = age_anat_pheno.drop('Age', axis=1)
        y = age_anat_pheno['Age']
        train_meta_X, test_meta_X, train_meta_y, test_meta_y = train_test_split(X, y, test_size=0.2, random_state=seed)
        
        train_X, val_X, train_y, val_y = train_test_split(train_meta_X, train_meta_y, test_size=0.2, random_state=seed)
        
        K_ratio = k / test_meta_X.shape[0] - 0.005
        final_test_X, k_shot_X, final_test_y, k_shot_y = train_test_split(test_meta_X, test_meta_y, test_size=K_ratio, random_state=seed)
        
        
        # print(f"train_X.shape : {train_X.shape}")
        # print(f"val_X.shape : {val_X.shape}")
        # print(f"k_shot_X.shape : {k_shot_X.shape}")
        # print(f"final_test_X.shape : {final_test_X.shape}")
        
        train_subject = train_X.iloc[:, 0]
        train_anat = train_X.iloc[:, 1:anat_col_num+1]
        train_pheno = train_X.iloc[:, anat_col_num+1:]
        train_pheno = pd.concat([train_pheno, train_y], axis=1)
        
        val_subject = val_X.iloc[:, 0]
        val_anat = val_X.iloc[:, 1:anat_col_num+1]
        val_pheno = val_X.iloc[:, anat_col_num+1:]
        val_pheno = pd.concat([val_pheno, val_y], axis=1)
        
        k_shot_subject = k_shot_X.iloc[:, 0]
        k_shot_anat = k_shot_X.iloc[:, 1:anat_col_num+1]
        k_shot_pheno = k_shot_X.iloc[:, anat_col_num+1:]
        k_shot_pheno = pd.concat([k_shot_pheno, k_shot_y], axis=1)
        
        final_test_subject = final_test_X.iloc[:, 0]
        final_test_anat = final_test_X.iloc[:, 1:anat_col_num+1]
        final_test_pheno = final_test_X.iloc[:, anat_col_num+1:]
        final_test_pheno = pd.concat([final_test_pheno, final_test_y], axis=1)
        
        
        # Phenotype들을 Z-normalization 
        # Train meta(Train, Val)은 Train에서 mean, std를 계산해서 
        # Test meta(K, final_test)은 K에서 mean, std를 계산해서 
        # 이때 Age도 동시에 Normalization 수행한다. 
        train_meta_scaler = StandardScaler()
        test_meta_scaler = StandardScaler()
        
        train_meta_scaler.fit(train_pheno)
        train_pheno = pd.DataFrame(train_meta_scaler.transform(train_pheno))
        val_pheno = pd.DataFrame(train_meta_scaler.transform(val_pheno))
            
        test_meta_scaler.fit(k_shot_pheno)
        k_shot_pheno = pd.DataFrame(test_meta_scaler.transform(k_shot_pheno))
        final_test_pheno = pd.DataFrame(test_meta_scaler.transform(final_test_pheno))
        
        # KRR list 초기화 
        model_list = [model] * (pheno.shape[1])
        
        # 모델을 학습시키고, 다시 모델 리스트에 저장 
        for i, model in enumerate(model_list):
            model.fit(train_anat, train_pheno.iloc[:, i])
            model_list[i] = model 
            
        train_corr_list = []
        val_corr_list = []
        
        # 이것은 Age를 예측하는 것과는 별개로, 모델이 제대로 학습이 되었는지를 판단하기 위해서 
        # 동일한 Phenotype을 가지고 Test sample을 만들어서 성능을 확인하는 것 
        for i, model in enumerate(model_list):
            train_pred = model.predict(train_anat)
            train_corr_list.append(np.corrcoef(train_pheno.iloc[:, i], train_pred)[0][1])
            
            val_pred = model.predict(val_anat)
            val_corr_list.append(np.corrcoef(val_pheno.iloc[:, i], val_pred)[0][1])
            
        
        # 33개의 Phenotype을 예측하는 것에 있어서 평균 Correlation을 Print
        # print(f"Average Train correlation accuracy : {round(sum(train_corr_list) / len(train_corr_list),3)}")
        # print(f"Average Valid correlation accuracy :{round(sum(val_corr_list) / len(val_corr_list),3)}")
        
        
        
        # 33개의 KRR을 가지고 Age를 가장 잘 예측하는 KRR을 선택 
        best_corr = -100
        best_model_idx = -1
        
        # 33 KRR을 돌아가면서 k_shot에 대해서 Age prediction을 진행 
        for i, model in enumerate(model_list):
            model.fit(train_anat, train_pheno.iloc[:, i])
            pred = model.predict(k_shot_anat)
            k_shot_corr = np.corrcoef(k_shot_y, pred)[0][1]
            # print(f"{i}th model k_shot_corr : {k_shot_corr}")
            
            if best_corr < k_shot_corr:
                best_model_idx = i
                best_corr = k_shot_corr
                
        best_model = model_list[best_model_idx]
        best_model.fit(train_anat, train_pheno.iloc[:, best_model_idx])
        best_model_k_shot_metric = np.corrcoef(k_shot_y, best_model.predict(k_shot_anat))
        
        final_pred = best_model.predict(final_test_anat)
        final_corr = round(np.corrcoef(final_test_y, final_pred)[0][1], 4)
        final_corr_list.append(final_corr)
        
        # print("best model idx : ", best_model_idx)
        # print(f"best_model이 k-shot을 예측한 성능 : {best_model_k_shot_metric[0][1]}")
        # print("final_corr : ", final_corr[0][1])
        
    
    final_corr_np = np.array(final_corr_list)
    mean = np.mean(final_corr_np)
    std = np.std(final_corr_np)
    
    print(f"final correlation mean : {mean}")
    print(f"final correlation std : {std}")
    print(f"final correlation list : {final_corr_list}")
    
    print('\n\n===========================================\n\n')

### Basic DNN

In [14]:
def basic_dnn_exp(anat_name ,anat, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        # Random Seed 설정
        print('\n\n iter ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        # 모델 가중치 파일 이름 설정
        model_pth = anat_name + '_'+str(seed) +'_k'+ str(k_num) +'_dnn4l.pth'
        
        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        meta_x_data, meta_y_data = make_meta(anat, pheno_with_age, subject_list, seed)        
        train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta = dataset_preprocessing(meta_x_data, meta_y_data, k_num, seed)
        
        # train
        train_meta_x = torch.Tensor(train_meta_x).to(device)
        train_meta_y = torch.Tensor(train_meta_y).to(device)
        # validation
        val_meta_x = torch.Tensor(val_meta_x).to(device)
        val_meta_y = torch.Tensor(val_meta_y).to(device)
        # k-shot
        k_train_meta = torch.Tensor(k_train_meta).to(device)
        k_val_meta = torch.Tensor(k_val_meta).to(device)
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        rest_train_meta = torch.Tensor(rest_train_meta).to(device)
        rest_val_meta = torch.Tensor(rest_val_meta).to(device)
        
        train_dataset = torch.utils.data.TensorDataset(train_meta_x, train_meta_y)
        val_dataset = torch.utils.data.TensorDataset(val_meta_x, val_meta_y)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
        k_train_set = torch.utils.data.TensorDataset(k_train_meta, k_val_meta)
        rest_train_set = torch.utils.data.TensorDataset(rest_train_meta, rest_val_meta)
        k_train_dataloader = DataLoader(k_train_set, batch_size = k_train_meta.shape[0], shuffle=True, drop_last=True)
        rest_train_dataloader = DataLoader(rest_train_set, batch_size=16, shuffle=True)
        


        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_meta_x.shape[1], 256, 128, 64, 0.2, train_meta_y.shape[1]).to(device)
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                    
            # Epoch 10 단위마다 출력 
            # if epoch % 10 == 9:
                # print(f"Epoch {epoch+1} - Train Loss: {loss:.4f}")
                
            # if epoch == 99:
                # torch.save(model, model_pth)  
        
        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        output_mat = val_outputs.T
        validation_mat = val_meta_y.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(output_mat)):
            val_pheno_corrs.append(np.corrcoef(output_mat[i], validation_mat[i])[0][1]) 
            
        print("=====Validation=====")
        print('correlation :',sum(val_pheno_corrs)/len(val_pheno_corrs))
        print('max correlation index :', val_pheno_corrs.index(max(val_pheno_corrs)))
        
        
        # k shot Prediction
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_train_dataloader:
                K_outputs = model(inputs)
                k_train_outputs.append(K_outputs)
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_shot_age_corrs = []
        k_train_outputs = k_train_outputs.T
        for i in range(len(k_train_outputs)):
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_test_meta)[0][1]) 
        
        print("=====K-shot=====")
        print('k-shot correlation :',sum(k_shot_age_corrs)/len(k_shot_age_corrs))
        print('k-shot max correlation index :', k_shot_age_corrs.index(max(k_shot_age_corrs)))
            
        # k-shot max correlation index 
        max_corr_pheno_index = k_shot_age_corrs.index(max(k_shot_age_corrs))

        
        # rest Prediction
        rest_outputs = [] 
        with torch.no_grad():
            for inputs, targets in rest_train_dataloader:
                outputs = model(inputs)
                rest_outputs.append(outputs)
        rest_outputs = torch.cat(rest_outputs).cpu().detach().numpy()
        # k shot에서 max correlation index에 해당하는 output만 가져옴
        rest_outputs = rest_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        
        age_corr = np.corrcoef(rest_outputs[:, 0], rest_test_meta)[0][1]
        print("=====Test=====")
        print('age corr :', age_corr)
        corres.append(age_corr)
        
    if iteration > 1 : 
        print(np.mean(corres))
        print(np.std(corres))        

### Advanced Stacking

In [15]:
def get_top_k_values_with_indices(k_shot_age_corrs, k_num):
    indexed_lst = list(enumerate(k_shot_age_corrs))  # 리스트의 값과 해당 값의 인덱스를 함께 저장한 리스트 생성
    sorted_lst = sorted(indexed_lst, key=lambda x: x[1], reverse=True)  # 값에 대해 내림차순으로 정렬
    top_k_values = sorted_lst[:k_num]  # 상위 K개의 값만 선택
    top_k_indices = [x[0] for x in top_k_values]  # 선택된 값들의 인덱스 추출
    return top_k_indices

In [16]:
def advanced_stacking(anat_name ,anat, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        # Random Seed 설정
        print('\n\n iter ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        # 모델 가중치 파일 이름 설정
        model_pth = anat_name + '_'+str(seed) +'_k'+ str(k_num) +'_dnn4l.pth'
        
        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        meta_x_data, meta_y_data = make_meta(anat, pheno_with_age, subject_list, seed)        
        train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta = dataset_preprocessing(meta_x_data, meta_y_data, k_num, seed)
        
        # train
        train_meta_x = torch.Tensor(train_meta_x).to(device)
        train_meta_y = torch.Tensor(train_meta_y).to(device)
        # validation
        val_meta_x = torch.Tensor(val_meta_x).to(device)
        val_meta_y = torch.Tensor(val_meta_y).to(device)
        # k-shot
        k_train_meta = torch.Tensor(k_train_meta).to(device)
        k_val_meta = torch.Tensor(k_val_meta).to(device)
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        rest_train_meta = torch.Tensor(rest_train_meta).to(device)
        rest_val_meta = torch.Tensor(rest_val_meta).to(device)
        
        train_dataset = torch.utils.data.TensorDataset(train_meta_x, train_meta_y)
        val_dataset = torch.utils.data.TensorDataset(val_meta_x, val_meta_y)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
        k_train_set = torch.utils.data.TensorDataset(k_train_meta, k_val_meta)
        rest_train_set = torch.utils.data.TensorDataset(rest_train_meta, rest_val_meta)
        k_train_dataloader = DataLoader(k_train_set, batch_size = k_train_meta.shape[0], shuffle=True, drop_last=True)
        rest_train_dataloader = DataLoader(rest_train_set, batch_size=16, shuffle=True)
        


        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_meta_x.shape[1], 256, 128, 64, 0.2, train_meta_y.shape[1]).to(device)
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                    
            # Epoch 10 단위마다 출력 
            # if epoch % 10 == 9:
                # print(f"Epoch {epoch+1} - Train Loss: {loss:.4f}")
                
            # if epoch == 99:
                # torch.save(model, model_pth)  
        
        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        output_mat = val_outputs.T
        validation_mat = val_meta_y.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(output_mat)):
            val_pheno_corrs.append(np.corrcoef(output_mat[i], validation_mat[i])[0][1]) 
            
        print("=====Validation=====")
        print('correlation :',sum(val_pheno_corrs)/len(val_pheno_corrs))
        print('max correlation index :', val_pheno_corrs.index(max(val_pheno_corrs)))
        
        
        
        # k shot Prediction
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_train_dataloader:
                K_outputs = model(inputs)
                k_train_outputs.append(K_outputs)
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_shot_age_corrs = []
        k_train_outputs = k_train_outputs.T
        for i in range(len(k_train_outputs)):
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_test_meta)[0][1]) 
        
        # K-shot에서 가장 correlation이 높은 k개 features를 선정  
        top_k_indices = get_top_k_values_with_indices(k_shot_age_corrs, k_num)
        
        top_k_train_outputs = k_train_outputs[top_k_indices].T
        # K개의 Prediction들이 KRR을 학습 시키는데에 features로 사용된다. 
        
        kf = KFold(n_splits=5, shuffle=True, random_state=seed)  # 5-fold cross validation
        # 모델 초기화
        krr_model = KernelRidge()
        
        # 교차 검증 수행
        for train_index, val_index in kf.split(top_k_train_outputs):
            X_train, X_val = top_k_train_outputs[train_index], top_k_train_outputs[val_index]
            y_train, y_val = k_test_meta[train_index], k_test_meta[val_index]

            # 모델 학습
            krr_model.fit(X_train, y_train)

            # 검증 데이터에 대한 예측
            y_pred = krr_model.predict(X_val)
        
        
        # rest Prediction
        rest_outputs = [] 
        with torch.no_grad():
            for inputs, targets in rest_train_dataloader:
                outputs = model(inputs)
                rest_outputs.append(outputs)
        rest_outputs = torch.cat(rest_outputs).cpu().detach().numpy()
        
        # k shot에서 top_k_indices에 해당하는 output만 가져옴
        top_k_rest_outputs = rest_outputs[:, top_k_indices]
        
        
        
        # Prediction
        rest_outputs = krr_model.predict(top_k_rest_outputs)
        
        age_corr = np.corrcoef(rest_outputs, rest_test_meta)[0][1]
        print("=====Test=====")
        print(f'age corr : {age_corr:.4f}')
        corres.append(age_corr)
        
    if iteration > 1 : 
        print(np.mean(corres))
        print(np.std(corres)) 
        
    print("===========================================================")

### Advanced Fine-Tuning

In [17]:
def advanced_finetune(anat_name ,anat, k_num, iteration = 10):
    
    corres = []
    
    # 반복 실험 
    for seed in range(1, iteration+1):
        
        
        # Random Seed 설정
        print('\n\n iter ', seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)        
        # CUDA를 사용하는 경우 => 이거 반복적으로 동일한 결과가 나오는 지 확인 
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

        # 모델 가중치 파일 이름 설정
        # model_pth = anat_name + '_'+str(seed) +'_k'+ str(k_num) +'_dnn4l.pth'
        
        # Anat과 pheno_with_age의 index를 랜덤하게 Shuffling -> Train meta set과 Test meta set으로 분리 
        meta_x_data, meta_y_data = make_meta(anat, pheno_with_age, subject_list, seed)        
        train_meta_x, val_meta_x, train_meta_y, val_meta_y, k_train_meta, rest_train_meta, k_val_meta, rest_val_meta, k_test_meta, rest_test_meta, ft_train_x, ft_val_x, ft_train_y, ft_val_y, ft_age_x, ft_age_y = dataset_finetune_preprocessing(meta_x_data, meta_y_data, k_num, seed)
        
        print(val_meta_x.shape, val_meta_y.shape)
        # train
        train_meta_x = torch.Tensor(train_meta_x).to(device)
        train_meta_y = torch.Tensor(train_meta_y).to(device)
        
        # validation
        val_meta_x = torch.Tensor(val_meta_x).to(device)
        val_meta_y = torch.Tensor(val_meta_y).to(device)
        
        # k-shot
        k_train_meta = torch.Tensor(k_train_meta).to(device)
        k_val_meta = torch.Tensor(k_val_meta).to(device)
        
        # Rest (Test meta set에서 K-shot을 제외한 나머지 samples)
        rest_train_meta = torch.Tensor(rest_train_meta).to(device)
        rest_val_meta = torch.Tensor(rest_val_meta).to(device)
        
        # fine tune train
        ft_train_x = torch.Tensor(ft_train_x).to(device)
        ft_train_y = torch.Tensor(ft_train_y).to(device)
        
        # fine tune validation
        ft_val_x = torch.Tensor(ft_val_x).to(device)
        ft_val_y = torch.Tensor(ft_val_y).to(device)        
        
        train_dataset = torch.utils.data.TensorDataset(train_meta_x, train_meta_y)
        val_dataset = torch.utils.data.TensorDataset(val_meta_x, val_meta_y)
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
        val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        k_train_set = torch.utils.data.TensorDataset(k_train_meta, k_val_meta)
        rest_train_set = torch.utils.data.TensorDataset(rest_train_meta, rest_val_meta)
        k_train_dataloader = DataLoader(k_train_set, batch_size = k_train_meta.shape[0], shuffle=True, drop_last=True)
        rest_train_dataloader = DataLoader(rest_train_set, batch_size=16, shuffle=True)
        
        ft_train_dataset = torch.utils.data.TensorDataset(ft_train_x, ft_train_y)
        ft_val_dataset = torch.utils.data.TensorDataset(ft_val_x, ft_val_y)
        ft_train_dataloader = DataLoader(ft_train_dataset, batch_size= ft_train_x.shape[0], shuffle=True, drop_last=True)
        ft_val_dataloader = DataLoader(ft_val_dataset, batch_size=16, shuffle=False)

        # 모델 정의 및 Loss function & Optimizer
        model = dnn_4l(train_meta_x.shape[1], 256, 128, 64, 0.2, train_meta_y.shape[1]).to(device)
        model.to(device)
        loss_function = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        
        # Meta train : Training (100 Epoch)
        for epoch in range(100):  
            model.train()
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                    
            # Epoch 10 단위마다 출력 
            # if epoch % 10 == 9:
                # print(f"Epoch {epoch+1} - Train Loss: {loss:.4f}")
                
            # if epoch == 99:
                # torch.save(model, model_pth)  
        
        
        # validation
        model.eval()
        # Validation prediction
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                outputs = model(inputs)
                val_outputs.append(outputs)
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        output_mat = val_outputs.T
        validation_mat = val_meta_y.detach().cpu().numpy().T
        
        # 긱 Phenotype마다, Correlation Coefficient를 저장하기 위한 리스트 
        val_pheno_corrs = []
        for i in range(len(output_mat)):
            val_pheno_corrs.append(np.corrcoef(output_mat[i], validation_mat[i])[0][1]) 
            
        print("===============Validation===============")
        print('correlation :',round(sum(val_pheno_corrs)/len(val_pheno_corrs),4))
        print('max correlation index :', val_pheno_corrs.index(max(val_pheno_corrs)))
        
        
        
        # k shot Prediction -> K 명에 대해서 Best prediction을 가지는 Node가 무엇인지를 판별 
        model.eval()
        k_train_outputs = []        
        with torch.no_grad():
            for inputs, targets in k_train_dataloader:
                K_outputs = model(inputs) # prediction
                k_train_outputs.append(K_outputs) # prediction list 저장 
        k_train_outputs = torch.cat(k_train_outputs).cpu().detach().numpy() # k_train_outputs에는 (k_num, 58)의 matrix가 형성? 
        k_shot_age_corrs = [] 
        k_train_outputs = k_train_outputs.T
        for i in range(len(k_train_outputs)):
            k_shot_age_corrs.append(np.corrcoef(k_train_outputs[i], k_test_meta)[0][1]) 
        
        print("===============K-shot===============")
        print('k-shot correlation :', round(sum(k_shot_age_corrs)/len(k_shot_age_corrs),4))
        print('k-shot max correlation index :', k_shot_age_corrs.index(max(k_shot_age_corrs)))    
        
        # k-shot max correlation index 
        max_corr_pheno_index = k_shot_age_corrs.index(max(k_shot_age_corrs))
        print('max corr pheno index', max_corr_pheno_index)
        
        
        # k shot Fine Tuning train
        # last 2 layers update
        # optimizer setting 
        optimizer = torch.optim.SGD([{'params': model.fc3.parameters()}, {'params': model.fc4.parameters()}],
                                    lr = 0.01, momentum = 0.9)
        for epoch in range(100):  
            model.train()
            for inputs, targets in ft_train_dataloader:
                optimizer.zero_grad()
                outputs = model(inputs) # (batch_size, 58)
                loss = loss_function(outputs, targets)
                loss.backward()
                optimizer.step()
                # 논문에서는 last 2 layer라고 되어 있는데, 여기에서는 그냥 전부를 다시 학습시키는 것아닌가? 
                    
        # k shot Fine Tuning validation
        model.eval()
        val_outputs = []
        with torch.no_grad():
            for inputs, targets in ft_val_dataloader:
                outputs = model(inputs) # validation prediction
                val_outputs.append(outputs) # validation prediction list 저장 
        val_outputs = torch.cat(val_outputs).cpu().detach().numpy()
        val_outputs = val_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        age_corr = np.corrcoef(val_outputs[:, 0], ft_age_y)[0][1]
        print("===============K-shot fine tuning validation===============")        
        print('age corr', round(age_corr,4))


        # rest Prediction
        rest_outputs = [] 
        with torch.no_grad():
            for inputs, targets in rest_train_dataloader:
                outputs = model(inputs)
                rest_outputs.append(outputs)
        rest_outputs = torch.cat(rest_outputs).cpu().detach().numpy()
        # k shot에서 max correlation index에 해당하는 output만 가져옴
        rest_outputs = rest_outputs[:, max_corr_pheno_index].reshape(-1, 1)
        age_corr = np.corrcoef(rest_outputs[:, 0], rest_test_meta)[0][1]
        print("===============Test===============")
        print('age corr :', round(age_corr,4), '\n\n')
        corres.append(age_corr)            
        
    if iteration > 1 : 
        print(np.mean(corres))
        print(np.std(corres)) 
        
    print("===========================================================\n\n")

# Experiments

### Basic KRR

**sMRI**

In [18]:
basic_krr(age_file=age_file, anat=cort_thick, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=cort_thick, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=cort_thick, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.007990000000000004
final correlation std : 0.15308134732879772
final correlation list : [-0.0718, 0.0485, 0.2442, -0.2005, 0.2405, -0.1529, -0.0983, -0.1354, 0.1161, -0.0703]






 Experiment : k=30, iteration=10
final correlation mean : 0.013419999999999998
final correlation std : 0.08713352741625924
final correlation list : [-0.0136, 0.1092, -0.0481, -0.1854, 0.0792, 0.0145, 0.0888, -0.0329, 0.1159, 0.0066]






 Experiment : k=10, iteration=10
final correlation mean : 0.02172
final correlation std : 0.04804004163195531
final correlation list : [0.0315, 0.1187, 0.0368, 0.0142, 0.0004, -0.0449, -0.0388, 0.0664, -0.0202, 0.0531]






In [19]:
basic_krr(age_file=age_file, anat=cort_area, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=cort_area, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=cort_area, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.004960000000000001
final correlation std : 0.08812868091603324
final correlation list : [-0.1118, 0.0609, -0.046, -0.1932, -0.0324, 0.0396, 0.1186, 0.0132, 0.0395, 0.062]






 Experiment : k=30, iteration=10
final correlation mean : 0.00707
final correlation std : 0.0832244441255092
final correlation list : [-0.1348, -0.0604, -0.0043, 0.1806, 0.0179, 0.0981, -0.0432, 0.0525, -0.0251, -0.0106]






 Experiment : k=10, iteration=10
final correlation mean : 0.006529999999999999
final correlation std : 0.06633031056764321
final correlation list : [0.0575, -0.1526, 0.0392, -0.0037, 0.023, 0.1138, -0.0126, -0.0207, -0.0208, 0.0422]






In [20]:
basic_krr(age_file=age_file, anat=cort_vol, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=cort_vol, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=cort_vol, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.07488000000000002
final correlation std : 0.08597223737928424
final correlation list : [-0.0799, 0.0818, -0.1819, -0.0502, -0.0267, -0.2292, -0.0766, 0.0172, -0.1264, -0.0769]






 Experiment : k=30, iteration=10
final correlation mean : 0.025949999999999994
final correlation std : 0.08436503126295869
final correlation list : [-0.0358, 0.1189, 0.1485, -0.0314, -0.046, -0.0805, 0.0176, -0.0173, 0.1735, 0.012]






 Experiment : k=10, iteration=10
final correlation mean : 0.04004
final correlation std : 0.09113286125213012
final correlation list : [-0.0503, 0.1198, 0.1847, 0.0518, -0.0485, 0.0487, -0.0165, 0.1913, -0.0235, -0.0571]






**TBSS**

In [21]:
basic_krr(age_file=age_file, anat=tbss_ad, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_ad, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_ad, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.00871
final correlation std : 0.08667661103204255
final correlation list : [0.0471, -0.0848, 0.0746, 0.0475, -0.0875, 0.0869, 0.0792, -0.0529, -0.1421, 0.1191]






 Experiment : k=30, iteration=10
final correlation mean : 0.004330000000000001
final correlation std : 0.08313722451465409
final correlation list : [-0.14, -0.0339, 0.0764, 0.0113, -0.0218, 0.011, 0.1494, 0.1075, -0.084, -0.0326]






 Experiment : k=10, iteration=10
final correlation mean : -0.002880000000000005
final correlation std : 0.10841922154304559
final correlation list : [0.0841, -0.0892, -0.1114, -0.0076, 0.0114, -0.134, 0.1112, 0.0578, -0.1416, 0.1905]






In [22]:
basic_krr(age_file=age_file, anat=tbss_fa, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_fa, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_fa, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.05697000000000001
final correlation std : 0.14146856223203796
final correlation list : [-0.0451, -0.0444, 0.0292, 0.2496, 0.0502, 0.0806, 0.0636, 0.003, -0.1689, 0.3519]






 Experiment : k=30, iteration=10
final correlation mean : 0.030890000000000008
final correlation std : 0.13326131809343625
final correlation list : [-0.0498, -0.0468, 0.11, -0.0558, 0.1082, 0.0786, 0.0644, -0.0831, -0.1549, 0.3381]






 Experiment : k=10, iteration=10
final correlation mean : -0.0017399999999999998
final correlation std : 0.1260099615109853
final correlation list : [0.0251, 0.1328, -0.1202, 0.0432, -0.1008, -0.0271, 0.1113, -0.0731, -0.2229, 0.2143]






In [23]:
basic_krr(age_file=age_file, anat=tbss_icvf, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_icvf, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_icvf, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.043829999999999994
final correlation std : 0.033534312278619936
final correlation list : [0.0328, 0.0559, 0.02, 0.0769, 0.0051, 0.0471, 0.0503, 0.0271, 0.1201, 0.003]






 Experiment : k=30, iteration=10
final correlation mean : 0.003580000000000001
final correlation std : 0.08479880659537609
final correlation list : [-0.0899, -0.0934, 0.0598, 0.0677, -0.0893, 0.053, 0.0709, 0.0631, 0.1137, -0.1198]






 Experiment : k=10, iteration=10
final correlation mean : -0.0010999999999999996
final correlation std : 0.059972460346395653
final correlation list : [0.0088, 0.0387, 0.0576, -0.0502, -0.0183, 0.0195, 0.0909, -0.0957, 0.034, -0.0963]






In [24]:
basic_krr(age_file=age_file, anat=tbss_isovf, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_isovf, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_isovf, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.038
final correlation std : 0.07514758811831555
final correlation list : [0.0957, 0.0972, 0.1831, 0.0184, -0.0553, -0.0428, 0.0495, -0.0174, 0.0946, -0.043]






 Experiment : k=30, iteration=10
final correlation mean : -0.017570000000000006
final correlation std : 0.09158146155199752
final correlation list : [0.0252, 0.0482, 0.0502, 0.0564, -0.1168, 0.0012, -0.2362, 0.0117, -0.0734, 0.0578]






 Experiment : k=10, iteration=10
final correlation mean : -0.049879999999999994
final correlation std : 0.08390142787819525
final correlation list : [-0.0097, -0.1627, -0.1184, 0.0723, -0.1066, 0.0072, -0.1724, 0.0675, -0.059, -0.017]






In [25]:
basic_krr(age_file=age_file, anat=tbss_md, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_md, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_md, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.031220000000000005
final correlation std : 0.11260869238207147
final correlation list : [-0.1772, 0.0417, 0.1169, -0.0557, -0.2283, 0.0033, 0.0576, -0.0048, -0.1533, 0.0876]






 Experiment : k=30, iteration=10
final correlation mean : -0.0008999999999999966
final correlation std : 0.0759460071366494
final correlation list : [-0.1939, 0.0264, -0.0252, -0.0078, -0.0074, 0.0305, 0.1137, 0.0251, 0.0579, -0.0283]






 Experiment : k=10, iteration=10
final correlation mean : -0.022969999999999997
final correlation std : 0.07118931169775418
final correlation list : [0.1247, -0.1244, 0.0297, -0.0807, -0.0231, -0.1281, -0.0083, 0.009, -0.01, -0.0185]






In [26]:
basic_krr(age_file=age_file, anat=tbss_od, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_od, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_od, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.06097
final correlation std : 0.08605636583077396
final correlation list : [-0.0599, 0.0373, 0.1408, -0.0177, 0.1327, 0.0243, 0.0106, 0.0347, 0.0547, 0.2522]






 Experiment : k=30, iteration=10
final correlation mean : -0.006489999999999996
final correlation std : 0.10707825596263697
final correlation list : [-0.0412, -0.0771, 0.1215, 0.0674, 0.1758, -0.1166, 0.0059, -0.1073, -0.1644, 0.0711]






 Experiment : k=10, iteration=10
final correlation mean : -0.0007899999999999984
final correlation std : 0.0577389028298945
final correlation list : [0.0263, 0.0142, -0.0235, -0.1063, 0.0924, 0.0244, -0.0579, -0.0165, 0.079, -0.04]






In [27]:
basic_krr(age_file=age_file, anat=tbss_rd, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tbss_rd, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tbss_rd, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.019639999999999998
final correlation std : 0.0733128256173502
final correlation list : [-0.107, 0.0132, -0.0491, -0.019, 0.057, -0.0383, 0.1092, 0.0387, -0.0513, -0.1498]






 Experiment : k=30, iteration=10
final correlation mean : 0.02443
final correlation std : 0.09755518489552466
final correlation list : [0.101, 0.0023, 0.0653, -0.0075, -0.0109, 0.1228, 0.1357, 0.0387, 0.0227, -0.2258]






 Experiment : k=10, iteration=10
final correlation mean : 0.025650000000000006
final correlation std : 0.09127554163082244
final correlation list : [-0.0004, -0.01, -0.04, 0.0314, 0.1443, 0.1425, -0.0404, -0.1556, 0.0525, 0.1322]






**Tractography**

In [28]:
basic_krr(age_file=age_file, anat=tractography_ad, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_ad, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_ad, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.04387999999999999
final correlation std : 0.07337561992923809
final correlation list : [-0.0846, -0.088, -0.0858, 0.0369, -0.0786, -0.0137, -0.0654, 0.1312, -0.1367, -0.0541]






 Experiment : k=30, iteration=10
final correlation mean : 0.011989999999999997
final correlation std : 0.08857955125196786
final correlation list : [0.1924, -0.0294, 0.0094, 0.025, -0.1189, 0.0509, -0.0629, -0.0784, 0.0121, 0.1197]






 Experiment : k=10, iteration=10
final correlation mean : -0.011660000000000004
final correlation std : 0.09522276198472716
final correlation list : [-0.0625, -0.0562, -0.0387, -0.1971, -0.0626, 0.0785, -0.0407, 0.0089, 0.1096, 0.1442]






In [29]:
basic_krr(age_file=age_file, anat=tractography_fa, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_fa, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_fa, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.0010500000000000036
final correlation std : 0.06557030196666781
final correlation list : [0.0355, -0.0671, 0.0772, 0.0958, 0.0267, 0.0242, -0.1015, -0.0657, -0.0705, 0.0349]






 Experiment : k=30, iteration=10
final correlation mean : -0.02118
final correlation std : 0.08667399610032989
final correlation list : [0.0402, -0.0893, 0.1309, 0.0815, -0.0235, -0.1111, -0.0824, -0.0286, -0.1579, 0.0284]






 Experiment : k=10, iteration=10
final correlation mean : -0.009320000000000004
final correlation std : 0.08858945535446078
final correlation list : [0.0019, 0.0612, -0.0753, -0.1365, 0.107, -0.067, -0.0809, 0.0219, 0.1503, -0.0758]






In [30]:
basic_krr(age_file=age_file, anat=tractography_icvf, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_icvf, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_icvf, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.05534
final correlation std : 0.12632102121183156
final correlation list : [-0.0113, 0.0253, -0.2001, 0.1619, 0.2279, 0.0169, 0.126, 0.1925, 0.1018, -0.0875]






 Experiment : k=30, iteration=10
final correlation mean : -0.012130000000000004
final correlation std : 0.08468969299743624
final correlation list : [-0.0975, 0.0595, -0.0585, 0.0906, 0.1542, -0.0782, 0.0061, -0.1114, -0.0782, -0.0079]






 Experiment : k=10, iteration=10
final correlation mean : 0.03934
final correlation std : 0.09582414309556855
final correlation list : [-0.053, -0.0115, -0.1035, 0.1411, 0.206, -0.0022, 0.0299, 0.1229, 0.1133, -0.0496]






In [31]:
# tbss_ad tbss_fa tbss_icvf tbss_isovf tbss_md tbss_od tbss_rd 
# tracto_ad tracto_fa tracto_icvf tracto_isovf tracto_md tracto_od tracto_rd 


basic_krr(age_file=age_file, anat=tractography_isovf, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_isovf, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_isovf, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.05471
final correlation std : 0.1262374702693301
final correlation list : [0.2082, -0.0252, -0.1211, 0.0391, -0.146, 0.159, 0.1588, 0.233, 0.0535, -0.0122]






 Experiment : k=30, iteration=10
final correlation mean : 0.024600000000000004
final correlation std : 0.09787627904655959
final correlation list : [0.0903, 0.0542, -0.1389, 0.041, -0.0634, -0.0355, 0.1071, 0.209, 0.0589, -0.0767]






 Experiment : k=10, iteration=10
final correlation mean : 0.023770000000000003
final correlation std : 0.06157408627011854
final correlation list : [0.0904, 0.0693, 0.003, -0.014, 0.0129, -0.0424, 0.1167, -0.0833, -0.0035, 0.0886]






In [32]:
# tbss_ad tbss_fa tbss_icvf tbss_isovf tbss_md tbss_od tbss_rd 
# tracto_ad tracto_fa tracto_icvf tracto_isovf tracto_md tracto_od tracto_rd 


basic_krr(age_file=age_file, anat=tractography_md, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_md, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_md, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.01982
final correlation std : 0.06987945048438775
final correlation list : [0.0095, -0.0712, -0.0029, 0.0652, -0.007, -0.122, -0.0373, 0.1209, -0.0589, -0.0945]






 Experiment : k=30, iteration=10
final correlation mean : -0.031560000000000005
final correlation std : 0.06841723759404497
final correlation list : [-0.0575, -0.113, 0.0013, -0.1534, 0.0326, -0.055, 0.1045, -0.0145, -0.0376, -0.023]






 Experiment : k=10, iteration=10
final correlation mean : 0.02187
final correlation std : 0.07416938789015318
final correlation list : [0.0697, -0.1057, -0.0009, 0.1232, -0.0383, 0.1565, -0.0143, -0.0012, 0.0463, -0.0166]






In [33]:
# tbss_ad tbss_fa tbss_icvf tbss_isovf tbss_md tbss_od tbss_rd 
# tracto_ad tracto_fa tracto_icvf tracto_isovf tracto_md tracto_od tracto_rd 


basic_krr(age_file=age_file, anat=tractography_od, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_od, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_od, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.07934
final correlation std : 0.07824236959601875
final correlation list : [0.1092, 0.1826, 0.1108, 0.0685, 0.0198, -0.0097, 0.1382, 0.0167, 0.2005, -0.0432]






 Experiment : k=30, iteration=10
final correlation mean : 0.044789999999999996
final correlation std : 0.10356265205178941
final correlation list : [-0.0936, 0.1864, 0.1524, -0.0521, 0.1066, 0.0008, -0.0819, -0.0162, 0.1913, 0.0542]






 Experiment : k=10, iteration=10
final correlation mean : 0.04952
final correlation std : 0.09981193115053931
final correlation list : [0.0577, 0.005, 0.0408, -0.0249, -0.0124, -0.043, 0.1726, -0.0841, 0.247, 0.1365]






In [34]:
# tbss_ad tbss_fa tbss_icvf tbss_isovf tbss_md tbss_od tbss_rd 
# tracto_ad tracto_fa tracto_icvf tracto_isovf tracto_md tracto_od tracto_rd 


basic_krr(age_file=age_file, anat=tractography_rd, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_rd, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_rd, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.0245
final correlation std : 0.09761354414219371
final correlation list : [0.2031, -0.0862, -0.0276, -0.0388, -0.153, -0.0352, -0.0423, 0.0776, -0.0092, -0.1334]






 Experiment : k=30, iteration=10
final correlation mean : 0.019119999999999995
final correlation std : 0.07630938081258425
final correlation list : [0.1069, 0.0832, 0.0116, -0.1416, -0.0238, 0.1064, -0.0231, -0.0545, 0.0653, 0.0608]






 Experiment : k=10, iteration=10
final correlation mean : 0.003330000000000001
final correlation std : 0.0770861861814424
final correlation list : [0.0737, 0.0883, -0.0586, -0.0697, -0.1249, 0.1396, -0.035, -0.0248, 0.0158, 0.0289]






In [35]:
# tbss_ad tbss_fa tbss_icvf tbss_isovf tbss_md tbss_od tbss_rd 
# tracto_ad tracto_fa tracto_icvf tracto_isovf tracto_md tracto_od tracto_rd tracto_fss tracto_fssl


basic_krr(age_file=age_file, anat=tractography_fss, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_fss, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_fss, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.05442
final correlation std : 0.06365352778911787
final correlation list : [0.0336, 0.0045, -0.0359, -0.0945, -0.1266, -0.0835, -0.1017, 0.0471, -0.1469, -0.0403]






 Experiment : k=30, iteration=10
final correlation mean : -0.005930000000000003
final correlation std : 0.08146447139704523
final correlation list : [-0.0527, -0.0691, -0.0927, 0.0686, -0.1552, 0.0028, 0.1066, 0.0871, 0.0526, -0.0073]






 Experiment : k=10, iteration=10
final correlation mean : 0.018520000000000002
final correlation std : 0.08642667180911226
final correlation list : [0.1273, 0.0598, -0.0633, 0.0848, -0.0978, 0.0205, 0.0296, 0.0757, 0.0952, -0.1466]






In [36]:
basic_krr(age_file=age_file, anat=tractography_fssl, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=tractography_fssl, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=tractography_fssl, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.0055999999999999965
final correlation std : 0.09337621752887616
final correlation list : [0.1012, -0.1036, -0.1028, 0.0988, 0.0866, 0.0118, 0.0772, 0.0769, -0.0294, -0.1607]






 Experiment : k=30, iteration=10
final correlation mean : -0.03855000000000001
final correlation std : 0.09404902179182939
final correlation list : [0.1086, -0.0586, -0.2154, -0.0747, -0.0559, 0.1303, -0.0609, -0.0022, -0.081, -0.0757]






 Experiment : k=10, iteration=10
final correlation mean : -0.03547
final correlation std : 0.04583948189061478
final correlation list : [0.005, -0.013, -0.0291, -0.0661, -0.1205, -0.0561, -0.049, -0.0675, -0.0159, 0.0575]






**fMRI**

In [37]:
# f_gamb f_lang f_motor f_rs f_social f_wm 


basic_krr(age_file=age_file, anat=f_gamb, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=f_gamb, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=f_gamb, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.049839999999999995
final correlation std : 0.11735453293332984
final correlation list : [0.1854, -0.0053, 0.0708, -0.0492, 0.1732, 0.0113, 0.0185, -0.1816, 0.0392, 0.2361]






 Experiment : k=30, iteration=10
final correlation mean : -0.01585000000000001
final correlation std : 0.08742136180591102
final correlation list : [-0.0935, -0.0617, -0.0636, -0.05, 0.2047, -0.0538, 0.0245, -0.0028, 0.0479, -0.1102]






 Experiment : k=10, iteration=10
final correlation mean : 0.0035299999999999997
final correlation std : 0.08510035311325095
final correlation list : [0.1039, 0.0037, 0.0057, -0.068, 0.1168, 0.0667, -0.0185, -0.1306, 0.0793, -0.1237]






In [38]:
# f_gamb f_lang f_motor f_rs f_social f_wm 


basic_krr(age_file=age_file, anat=f_lang, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=f_lang, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=f_lang, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.036070000000000005
final correlation std : 0.1534810089229283
final correlation list : [0.1703, 0.1086, -0.0105, 0.0777, 0.4004, -0.1377, -0.0174, -0.0293, -0.0794, -0.122]






 Experiment : k=30, iteration=10
final correlation mean : 0.04578
final correlation std : 0.1228895504101142
final correlation list : [0.172, -0.0712, -0.061, 0.1127, 0.3345, 0.0649, 0.0045, -0.0219, -0.0644, -0.0123]






 Experiment : k=10, iteration=10
final correlation mean : 0.007540000000000001
final correlation std : 0.05785409579277858
final correlation list : [0.0191, 0.1302, 0.0512, -0.0041, 0.0364, -0.0918, -0.0093, 0.0265, -0.0338, -0.049]






In [39]:
# f_gamb f_lang f_motor f_rs f_social f_wm 


basic_krr(age_file=age_file, anat=f_motor, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=f_motor, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=f_motor, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.005619999999999998
final correlation std : 0.08745177871261396
final correlation list : [0.1221, -0.0315, -0.0339, 0.0715, 0.1053, 0.0728, -0.1665, -0.028, -0.0918, 0.0362]






 Experiment : k=30, iteration=10
final correlation mean : 0.01379
final correlation std : 0.08967428226643356
final correlation list : [0.0958, 0.0071, -0.0664, 0.0985, 0.1657, 0.0425, 0.0526, -0.047, -0.0701, -0.1408]






 Experiment : k=10, iteration=10
final correlation mean : 0.014929999999999999
final correlation std : 0.09123818334447481
final correlation list : [-0.0348, -0.0478, -0.0834, -0.0337, 0.1054, 0.0779, 0.043, -0.1356, 0.0897, 0.1686]






In [40]:
# f_gamb f_lang f_motor f_rs f_social f_wm 


basic_krr(age_file=age_file, anat=f_rs, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=f_rs, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=f_rs, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.05875999999999999
final correlation std : 0.08463048150636979
final correlation list : [0.1018, 0.1289, 0.0728, -0.1587, 0.0596, 0.126, 0.1613, 0.0336, 0.0404, 0.0219]






 Experiment : k=30, iteration=10
final correlation mean : -0.006950000000000003
final correlation std : 0.10821463163546785
final correlation list : [-0.033, -0.0509, 0.1041, -0.1597, 0.0831, -0.1319, 0.1803, 0.0369, -0.1345, 0.0361]






 Experiment : k=10, iteration=10
final correlation mean : -0.026019999999999998
final correlation std : 0.11392297222246267
final correlation list : [0.0249, 0.0059, 0.1497, 0.0134, 0.0141, 0.0287, -0.2024, 0.0659, -0.2121, -0.1483]






In [41]:
# f_gamb f_lang f_motor f_rs f_social f_wm 


basic_krr(age_file=age_file, anat=f_social, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=f_social, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=f_social, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : -0.015769999999999996
final correlation std : 0.10293079276873368
final correlation list : [0.1235, 0.0347, 0.0075, 0.048, -0.0501, -0.2527, 0.0251, -0.0469, 0.0749, -0.1217]






 Experiment : k=30, iteration=10
final correlation mean : -0.028090000000000004
final correlation std : 0.08294508363971913
final correlation list : [-0.0301, -0.0525, -0.1072, 0.0291, -0.0543, 0.1308, -0.0009, -0.1811, -0.0695, 0.0548]






 Experiment : k=10, iteration=10
final correlation mean : 0.007120000000000003
final correlation std : 0.07349142535017265
final correlation list : [-0.0225, -0.0418, 0.0519, 0.0363, 0.0363, 0.0199, 0.0379, -0.1588, -0.0265, 0.1385]






In [42]:
# f_gamb f_lang f_motor f_rs f_social f_wm 


basic_krr(age_file=age_file, anat=f_wm, pheno=pheno_58, model=KernelRidge(), k=50, iteration=10)
basic_krr(age_file=age_file, anat=f_wm, pheno=pheno_58, model=KernelRidge(), k=30, iteration=10)
basic_krr(age_file=age_file, anat=f_wm, pheno=pheno_58, model=KernelRidge(), k=10, iteration=10)



 Experiment : k=50, iteration=10
final correlation mean : 0.047119999999999995
final correlation std : 0.10260550472562376
final correlation list : [0.0553, -0.1708, 0.0695, 0.1393, 0.0597, 0.1626, -0.0814, -0.0094, 0.0788, 0.1676]






 Experiment : k=30, iteration=10
final correlation mean : 0.043179999999999996
final correlation std : 0.09664177978493566
final correlation list : [0.1443, -0.0617, -0.0071, 0.1624, 0.0557, 0.172, -0.0658, -0.0388, -0.0645, 0.1353]






 Experiment : k=10, iteration=10
final correlation mean : 0.02569
final correlation std : 0.08337067170174414
final correlation list : [0.0756, 0.0095, 0.001, -0.0201, -0.0683, 0.1714, 0.0276, -0.0588, -0.0513, 0.1703]






### Basic DNN 

In [45]:
basic_dnn_exp('cort_thick' ,cort_thick, 10, 10)
basic_dnn_exp('cort_thick' ,cort_thick, 30, 10)
basic_dnn_exp('cort_thick' ,cort_thick, 50, 10)



 iter  1
=====Validation=====
correlation : -0.014741384830879037
max correlation index : 19
=====K-shot=====
k-shot correlation : -0.11697996170286545
k-shot max correlation index : 43
=====Test=====
age corr : -0.08097027124221777


 iter  2
=====Validation=====
correlation : 0.01297096198202649
max correlation index : 56
=====K-shot=====
k-shot correlation : -0.04364681177598718
k-shot max correlation index : 14
=====Test=====
age corr : 0.10437741452410694


 iter  3
=====Validation=====
correlation : 0.04608548408649847
max correlation index : 42
=====K-shot=====
k-shot correlation : -0.044319738113919
k-shot max correlation index : 34
=====Test=====
age corr : 0.08280535642014267


 iter  4
=====Validation=====
correlation : 0.008927787421286982
max correlation index : 3
=====K-shot=====
k-shot correlation : -0.03647026804521018
k-shot max correlation index : 21
=====Test=====
age corr : -0.04733096553492172


 iter  5
=====Validation=====
correlation : -0.01743479018071752
max

In [46]:
basic_dnn_exp('cort_area' ,cort_area, 10, 10)
basic_dnn_exp('cort_area' ,cort_area, 30, 10)
basic_dnn_exp('cort_area' ,cort_area, 50, 10)



 iter  1
=====Validation=====
correlation : 0.008881405000889681
max correlation index : 25
=====K-shot=====
k-shot correlation : -0.09124720489720799
k-shot max correlation index : 39
=====Test=====
age corr : 0.03712728785529856


 iter  2
=====Validation=====
correlation : 0.007050461039663522
max correlation index : 48
=====K-shot=====
k-shot correlation : 0.11617074180717565
k-shot max correlation index : 7
=====Test=====
age corr : 0.039949217959496554


 iter  3
=====Validation=====
correlation : 0.05732433669733479
max correlation index : 50
=====K-shot=====
k-shot correlation : -0.06966107599607571
k-shot max correlation index : 18
=====Test=====
age corr : 0.019301500809802027


 iter  4
=====Validation=====
correlation : -0.04651453634635952
max correlation index : 23
=====K-shot=====
k-shot correlation : 0.23054308296572631
k-shot max correlation index : 35
=====Test=====
age corr : -0.019534425954432378


 iter  5
=====Validation=====
correlation : -0.005594361711308584


In [47]:
basic_dnn_exp('cort_vol' ,cort_vol, 10, 10)
basic_dnn_exp('cort_vol' ,cort_vol, 30, 10)
basic_dnn_exp('cort_vol' ,cort_vol, 50, 10)



 iter  1
=====Validation=====
correlation : -0.005780914989273103
max correlation index : 9
=====K-shot=====
k-shot correlation : -0.09403054363390961
k-shot max correlation index : 20
=====Test=====
age corr : 0.07123419082340818


 iter  2
=====Validation=====
correlation : 0.044869698325960314
max correlation index : 9
=====K-shot=====
k-shot correlation : -0.008480904339089277
k-shot max correlation index : 7
=====Test=====
age corr : -0.04664730313030661


 iter  3
=====Validation=====
correlation : 0.036451967754080744
max correlation index : 52
=====K-shot=====
k-shot correlation : 0.1361786252810586
k-shot max correlation index : 49
=====Test=====
age corr : -0.02876002233024323


 iter  4
=====Validation=====
correlation : 0.0238379913169321
max correlation index : 25
=====K-shot=====
k-shot correlation : -0.02444853852790021
k-shot max correlation index : 42
=====Test=====
age corr : 0.009678429482576335


 iter  5
=====Validation=====
correlation : -0.010834059896724476
ma

In [48]:
basic_dnn_exp('tbss_ad' ,tbss_ad, 10, 10)
basic_dnn_exp('tbss_ad' ,tbss_ad, 30, 10)
basic_dnn_exp('tbss_ad' ,tbss_ad, 50, 10)



 iter  1
=====Validation=====
correlation : 0.010939357452840715
max correlation index : 35
=====K-shot=====
k-shot correlation : 0.053221706542834465
k-shot max correlation index : 40
=====Test=====
age corr : 0.07901778232673005


 iter  2
=====Validation=====
correlation : 0.043329761869076396
max correlation index : 19
=====K-shot=====
k-shot correlation : 0.031015019814982385
k-shot max correlation index : 19
=====Test=====
age corr : 0.05336441586633386


 iter  3
=====Validation=====
correlation : 0.009746291421655095
max correlation index : 24
=====K-shot=====
k-shot correlation : 0.03416886638622944
k-shot max correlation index : 6
=====Test=====
age corr : -0.1479774827166645


 iter  4
=====Validation=====
correlation : -0.0036463610913845938
max correlation index : 34
=====K-shot=====
k-shot correlation : 0.03061084509360634
k-shot max correlation index : 10
=====Test=====
age corr : 0.03317584659399767


 iter  5
=====Validation=====
correlation : 0.00484276772890719
max

In [49]:
basic_dnn_exp('tbss_fa' ,tbss_fa, 10, 10)
basic_dnn_exp('tbss_fa' ,tbss_fa, 30, 10)
basic_dnn_exp('tbss_fa' ,tbss_fa, 50, 10)



 iter  1
=====Validation=====
correlation : -0.014782185889469746
max correlation index : 9
=====K-shot=====
k-shot correlation : 0.17174961855535092
k-shot max correlation index : 26
=====Test=====
age corr : 0.15234981707174441


 iter  2
=====Validation=====
correlation : -0.010061916818203197
max correlation index : 22
=====K-shot=====
k-shot correlation : 0.002724007019619749
k-shot max correlation index : 23
=====Test=====
age corr : 0.13156067937732535


 iter  3
=====Validation=====
correlation : -0.013579966541132617
max correlation index : 33
=====K-shot=====
k-shot correlation : 0.11270205076728737
k-shot max correlation index : 0
=====Test=====
age corr : -0.12159109766584257


 iter  4
=====Validation=====
correlation : -0.01930258294706437
max correlation index : 20
=====K-shot=====
k-shot correlation : 0.001383244465798817
k-shot max correlation index : 54
=====Test=====
age corr : 0.09289209224170643


 iter  5
=====Validation=====
correlation : -0.03111530645655727
m

In [50]:
basic_dnn_exp('tbss_icvf' ,tbss_icvf, 10, 10)
basic_dnn_exp('tbss_icvf' ,tbss_icvf, 30, 10)
basic_dnn_exp('tbss_icvf' ,tbss_icvf, 50, 10)



 iter  1
=====Validation=====
correlation : 0.004982721460880043
max correlation index : 33
=====K-shot=====
k-shot correlation : -0.056035975175695925
k-shot max correlation index : 17
=====Test=====
age corr : 0.008472166973047117


 iter  2
=====Validation=====
correlation : 0.0019652241641651114
max correlation index : 38
=====K-shot=====
k-shot correlation : 0.13693907806134287
k-shot max correlation index : 29
=====Test=====
age corr : 0.0170167734344011


 iter  3
=====Validation=====
correlation : -0.03449288226628131
max correlation index : 37
=====K-shot=====
k-shot correlation : 0.03921475191768717
k-shot max correlation index : 18
=====Test=====
age corr : -0.0201986618483837


 iter  4
=====Validation=====
correlation : 0.03060810586832802
max correlation index : 49
=====K-shot=====
k-shot correlation : 0.2070280445902241
k-shot max correlation index : 45
=====Test=====
age corr : -0.07861663775977709


 iter  5
=====Validation=====
correlation : -0.005298630996238104
ma

In [51]:
basic_dnn_exp('tbss_isovf' ,tbss_isovf, 10, 10)
basic_dnn_exp('tbss_isovf' ,tbss_isovf, 30, 10)
basic_dnn_exp('tbss_isovf' ,tbss_isovf, 50, 10)



 iter  1
=====Validation=====
correlation : 0.018382648004242615
max correlation index : 50
=====K-shot=====
k-shot correlation : -0.07006593231589023
k-shot max correlation index : 42
=====Test=====
age corr : -0.1986995654918567


 iter  2
=====Validation=====
correlation : -0.01854196973006063
max correlation index : 46
=====K-shot=====
k-shot correlation : -0.11766683414204722
k-shot max correlation index : 33
=====Test=====
age corr : -0.10637801029408794


 iter  3
=====Validation=====
correlation : -0.03858847921024542
max correlation index : 15
=====K-shot=====
k-shot correlation : 0.11938736586931467
k-shot max correlation index : 30
=====Test=====
age corr : -0.009375739452147641


 iter  4
=====Validation=====
correlation : -0.002010663797197011
max correlation index : 37
=====K-shot=====
k-shot correlation : 0.0760642736743477
k-shot max correlation index : 49
=====Test=====
age corr : -0.05325020781083308


 iter  5
=====Validation=====
correlation : -0.02278053072106973

In [52]:
basic_dnn_exp('tbss_md' ,tbss_md, 10, 10)
basic_dnn_exp('tbss_md' ,tbss_md, 30, 10)
basic_dnn_exp('tbss_md' ,tbss_md, 50, 10)



 iter  1
=====Validation=====
correlation : -0.030508174039553304
max correlation index : 8
=====K-shot=====
k-shot correlation : -0.013666781079296876
k-shot max correlation index : 1
=====Test=====
age corr : 0.13514477885846585


 iter  2
=====Validation=====
correlation : 0.022852581348411463
max correlation index : 54
=====K-shot=====
k-shot correlation : -0.036882680452182064
k-shot max correlation index : 20
=====Test=====
age corr : 0.2002093494506265


 iter  3
=====Validation=====
correlation : -0.011525073343220825
max correlation index : 1
=====K-shot=====
k-shot correlation : 0.09122156695451634
k-shot max correlation index : 21
=====Test=====
age corr : -0.030248086087602995


 iter  4
=====Validation=====
correlation : 0.005965896733324204
max correlation index : 0
=====K-shot=====
k-shot correlation : -0.07552060502528421
k-shot max correlation index : 17
=====Test=====
age corr : 0.018339064126399296


 iter  5
=====Validation=====
correlation : -0.008979516402136067

In [53]:
basic_dnn_exp('tbss_od' ,tbss_od, 10, 10)
basic_dnn_exp('tbss_od' ,tbss_od, 30, 10)
basic_dnn_exp('tbss_od' ,tbss_od, 50, 10)



 iter  1
=====Validation=====
correlation : 0.003389397642732974
max correlation index : 39
=====K-shot=====
k-shot correlation : -0.14251288295408554
k-shot max correlation index : 28
=====Test=====
age corr : -0.0037409794753174327


 iter  2
=====Validation=====
correlation : 0.022576431116521047
max correlation index : 42
=====K-shot=====
k-shot correlation : 0.01624625124584046
k-shot max correlation index : 15
=====Test=====
age corr : -0.06486357517220023


 iter  3
=====Validation=====
correlation : 0.00859296204946641
max correlation index : 15
=====K-shot=====
k-shot correlation : 0.0615789693127699
k-shot max correlation index : 27
=====Test=====
age corr : -0.047390155521054494


 iter  4
=====Validation=====
correlation : 0.015983500129607267
max correlation index : 45
=====K-shot=====
k-shot correlation : 0.06027332467224295
k-shot max correlation index : 9
=====Test=====
age corr : -0.019726058140677762


 iter  5
=====Validation=====
correlation : 0.035671781364453384

In [54]:
basic_dnn_exp('tbss_rd' ,tbss_rd, 10, 10)
basic_dnn_exp('tbss_rd' ,tbss_rd, 30, 10)
basic_dnn_exp('tbss_rd' ,tbss_rd, 50, 10)



 iter  1
=====Validation=====
correlation : -0.0034006261922911725
max correlation index : 46
=====K-shot=====
k-shot correlation : 0.06682703380031474
k-shot max correlation index : 22
=====Test=====
age corr : -0.06629005141588411


 iter  2
=====Validation=====
correlation : -0.004787916765329703
max correlation index : 47
=====K-shot=====
k-shot correlation : 0.24477677562210184
k-shot max correlation index : 17
=====Test=====
age corr : 0.024308005158450125


 iter  3
=====Validation=====
correlation : 0.0008411595270710015
max correlation index : 40
=====K-shot=====
k-shot correlation : -0.07467829079295785
k-shot max correlation index : 33
=====Test=====
age corr : -0.0469870654306613


 iter  4
=====Validation=====
correlation : -0.015880680335727582
max correlation index : 7
=====K-shot=====
k-shot correlation : 0.012731090129300585
k-shot max correlation index : 22
=====Test=====
age corr : 0.01709157689577173


 iter  5
=====Validation=====
correlation : 0.0400410739796574

In [55]:
basic_dnn_exp('tractography_ad' ,tractography_ad, 10, 10)
basic_dnn_exp('tractography_ad' ,tractography_ad, 30, 10)
basic_dnn_exp('tractography_ad' ,tractography_ad, 50, 10)



 iter  1
=====Validation=====
correlation : 0.0017207556403976646
max correlation index : 35
=====K-shot=====
k-shot correlation : -0.1028287380148769
k-shot max correlation index : 28
=====Test=====
age corr : -0.038327264762411814


 iter  2
=====Validation=====
correlation : 0.029814296812952137
max correlation index : 43
=====K-shot=====
k-shot correlation : -0.04227575307968361
k-shot max correlation index : 7
=====Test=====
age corr : 0.061529325568503886


 iter  3
=====Validation=====
correlation : 0.002487582927518222
max correlation index : 39
=====K-shot=====
k-shot correlation : 0.06237397721799477
k-shot max correlation index : 0
=====Test=====
age corr : 0.013223588904161051


 iter  4
=====Validation=====
correlation : 0.013897352918319221
max correlation index : 13
=====K-shot=====
k-shot correlation : 0.14217360208714433
k-shot max correlation index : 8
=====Test=====
age corr : 0.048663927020995534


 iter  5
=====Validation=====
correlation : -0.006903344612757088


In [56]:
basic_dnn_exp('tractography_fa' ,tractography_fa, 10, 10)
basic_dnn_exp('tractography_fa' ,tractography_fa, 30, 10)
basic_dnn_exp('tractography_fa' ,tractography_fa, 50, 10)



 iter  1
=====Validation=====
correlation : -0.017057359245374527
max correlation index : 13
=====K-shot=====
k-shot correlation : -0.06847916537518
k-shot max correlation index : 16
=====Test=====
age corr : 0.006152301096376949


 iter  2
=====Validation=====
correlation : 0.027692605270867244
max correlation index : 47
=====K-shot=====
k-shot correlation : -0.07293186161199174
k-shot max correlation index : 33
=====Test=====
age corr : 0.10906055400135693


 iter  3
=====Validation=====
correlation : -0.054201281341605165
max correlation index : 13
=====K-shot=====
k-shot correlation : 0.05275170339146289
k-shot max correlation index : 44
=====Test=====
age corr : 0.10232999687614913


 iter  4
=====Validation=====
correlation : 0.004923776407977081
max correlation index : 14
=====K-shot=====
k-shot correlation : -0.029750932636392495
k-shot max correlation index : 14
=====Test=====
age corr : 0.0517660142748493


 iter  5
=====Validation=====
correlation : -3.897186580631046e-05


In [57]:
basic_dnn_exp('tractography_icvf' ,tractography_icvf, 10, 10)
basic_dnn_exp('tractography_icvf' ,tractography_icvf, 30, 10)
basic_dnn_exp('tractography_icvf' ,tractography_icvf, 50, 10)



 iter  1
=====Validation=====
correlation : -0.014637941216910979
max correlation index : 7
=====K-shot=====
k-shot correlation : -0.14241731069430408
k-shot max correlation index : 20
=====Test=====
age corr : 0.006021359826654494


 iter  2
=====Validation=====
correlation : 0.034430895137505885
max correlation index : 3
=====K-shot=====
k-shot correlation : 0.03658568250082712
k-shot max correlation index : 15
=====Test=====
age corr : 0.03829842621434597


 iter  3
=====Validation=====
correlation : 0.015784259328969936
max correlation index : 27
=====K-shot=====
k-shot correlation : -0.02891571855193379
k-shot max correlation index : 23
=====Test=====
age corr : -0.06500881469931453


 iter  4
=====Validation=====
correlation : 0.013023006939930037
max correlation index : 2
=====K-shot=====
k-shot correlation : 0.0028919087137836705
k-shot max correlation index : 13
=====Test=====
age corr : -0.21718539580226578


 iter  5
=====Validation=====
correlation : 4.798393538329202e-05

In [58]:
basic_dnn_exp('tractography_isovf' ,tractography_isovf, 10, 10)
basic_dnn_exp('tractography_isovf' ,tractography_isovf, 30, 10)
basic_dnn_exp('tractography_isovf' ,tractography_isovf, 50, 10)



 iter  1
=====Validation=====
correlation : 0.001077216016556625
max correlation index : 9
=====K-shot=====
k-shot correlation : 0.08004986975157807
k-shot max correlation index : 2
=====Test=====
age corr : -0.08770221484449729


 iter  2
=====Validation=====
correlation : 0.0053958063429736455
max correlation index : 50
=====K-shot=====
k-shot correlation : 0.1197304866203913
k-shot max correlation index : 16
=====Test=====
age corr : -0.05955391574290499


 iter  3
=====Validation=====
correlation : -0.018046253917763856
max correlation index : 45
=====K-shot=====
k-shot correlation : 0.11582757179611927
k-shot max correlation index : 31
=====Test=====
age corr : 0.0044511138871910615


 iter  4
=====Validation=====
correlation : -0.016949790731133643
max correlation index : 10
=====K-shot=====
k-shot correlation : -0.07115093380983194
k-shot max correlation index : 21
=====Test=====
age corr : -0.031159155826103638


 iter  5
=====Validation=====
correlation : 0.00599088916605671

In [59]:
basic_dnn_exp('tractography_md' ,tractography_md, 10, 10)
basic_dnn_exp('tractography_md' ,tractography_md, 30, 10)
basic_dnn_exp('tractography_md' ,tractography_md, 50, 10)



 iter  1
=====Validation=====
correlation : -7.377960255450054e-05
max correlation index : 36
=====K-shot=====
k-shot correlation : -0.061217303618179605
k-shot max correlation index : 30
=====Test=====
age corr : -0.08970284201467037


 iter  2
=====Validation=====
correlation : -0.0070055082730366
max correlation index : 10
=====K-shot=====
k-shot correlation : -0.0002849401629142968
k-shot max correlation index : 40
=====Test=====
age corr : -0.1070818009180064


 iter  3
=====Validation=====
correlation : 0.026325831580293512
max correlation index : 45
=====K-shot=====
k-shot correlation : 0.06018233159943201
k-shot max correlation index : 15
=====Test=====
age corr : -0.12572188177916974


 iter  4
=====Validation=====
correlation : 0.00786588901044727
max correlation index : 20
=====K-shot=====
k-shot correlation : 0.14698236581151491
k-shot max correlation index : 25
=====Test=====
age corr : 0.025026597779528457


 iter  5
=====Validation=====
correlation : -0.007781765236622

In [60]:
basic_dnn_exp('tractography_od' ,tractography_od, 10, 10)
basic_dnn_exp('tractography_od' ,tractography_od, 30, 10)
basic_dnn_exp('tractography_od' ,tractography_od, 50, 10)



 iter  1
=====Validation=====
correlation : -0.057923116816545175
max correlation index : 7
=====K-shot=====
k-shot correlation : 0.06223319926317243
k-shot max correlation index : 4
=====Test=====
age corr : -0.0737979433548592


 iter  2
=====Validation=====
correlation : -0.02697715824734204
max correlation index : 7
=====K-shot=====
k-shot correlation : 0.10574630796155166
k-shot max correlation index : 40
=====Test=====
age corr : -0.052896898270477354


 iter  3
=====Validation=====
correlation : -0.014145867567870956
max correlation index : 38
=====K-shot=====
k-shot correlation : 0.22461272216749884
k-shot max correlation index : 3
=====Test=====
age corr : -0.012817220140493822


 iter  4
=====Validation=====
correlation : -0.004995234464656152
max correlation index : 23
=====K-shot=====
k-shot correlation : 0.12770163289015174
k-shot max correlation index : 17
=====Test=====
age corr : 0.10128133162239794


 iter  5
=====Validation=====
correlation : 0.0048777395998109265
m

In [61]:
basic_dnn_exp('tractography_rd' ,tractography_rd, 10, 10)
basic_dnn_exp('tractography_rd' ,tractography_rd, 30, 10)
basic_dnn_exp('tractography_rd' ,tractography_rd, 50, 10)



 iter  1
=====Validation=====
correlation : -0.012398895440575621
max correlation index : 41
=====K-shot=====
k-shot correlation : 0.07850588338594974
k-shot max correlation index : 41
=====Test=====
age corr : 0.01683809281343047


 iter  2
=====Validation=====
correlation : -0.05294854505611155
max correlation index : 28
=====K-shot=====
k-shot correlation : 0.16126895941832659
k-shot max correlation index : 4
=====Test=====
age corr : 0.07230292799933505


 iter  3
=====Validation=====
correlation : -0.017873015295538995
max correlation index : 22
=====K-shot=====
k-shot correlation : 0.04461383181266548
k-shot max correlation index : 31
=====Test=====
age corr : -0.09196545926795888


 iter  4
=====Validation=====
correlation : 0.010853212678742861
max correlation index : 35
=====K-shot=====
k-shot correlation : 0.04830950669769482
k-shot max correlation index : 9
=====Test=====
age corr : 0.07447444327363878


 iter  5
=====Validation=====
correlation : -0.03752219513524653
max 

In [62]:
basic_dnn_exp('tractography_fss' ,tractography_fss, 10, 10)
basic_dnn_exp('tractography_fss' ,tractography_fss, 30, 10)
basic_dnn_exp('tractography_fss' ,tractography_fss, 50, 10)



 iter  1
=====Validation=====
correlation : -0.020955113843185755
max correlation index : 17
=====K-shot=====
k-shot correlation : -0.12068602218420837
k-shot max correlation index : 56
=====Test=====
age corr : -0.05491649755092332


 iter  2
=====Validation=====
correlation : 0.03501952129345636
max correlation index : 3
=====K-shot=====
k-shot correlation : -0.12786915389172043
k-shot max correlation index : 53
=====Test=====
age corr : -0.10427362897878127


 iter  3
=====Validation=====
correlation : -0.0022433208740563816
max correlation index : 46
=====K-shot=====
k-shot correlation : 0.0046445085044027334
k-shot max correlation index : 50
=====Test=====
age corr : 0.09004238575232329


 iter  4
=====Validation=====
correlation : -0.03320551335910481
max correlation index : 22
=====K-shot=====
k-shot correlation : -0.044676501280658774
k-shot max correlation index : 21
=====Test=====
age corr : -0.07846697016897294


 iter  5
=====Validation=====
correlation : 0.01871582441485

In [63]:
basic_dnn_exp('tractography_fssl' ,tractography_fssl, 10, 10)
basic_dnn_exp('tractography_fssl' ,tractography_fssl, 30, 10)
basic_dnn_exp('tractography_fssl' ,tractography_fssl, 50, 10)



 iter  1
=====Validation=====
correlation : 0.012429121122580085
max correlation index : 18
=====K-shot=====
k-shot correlation : 0.13844149834051322
k-shot max correlation index : 39
=====Test=====
age corr : -0.03193618860239013


 iter  2
=====Validation=====
correlation : -0.049611597369881015
max correlation index : 16
=====K-shot=====
k-shot correlation : 0.05398335135893004
k-shot max correlation index : 53
=====Test=====
age corr : 0.034345223722524985


 iter  3
=====Validation=====
correlation : -0.014102759388839845
max correlation index : 15
=====K-shot=====
k-shot correlation : -0.04736162642330114
k-shot max correlation index : 16
=====Test=====
age corr : -0.07103759196977015


 iter  4
=====Validation=====
correlation : 0.025348382694248297
max correlation index : 52
=====K-shot=====
k-shot correlation : 0.07886846555447576
k-shot max correlation index : 47
=====Test=====
age corr : -0.04300732082898115


 iter  5
=====Validation=====
correlation : 0.01951283176906904

In [64]:
basic_dnn_exp('f_gamb' ,f_gamb, 10, 10)
basic_dnn_exp('f_gamb' ,f_gamb, 30, 10)
basic_dnn_exp('f_gamb' ,f_gamb, 50, 10)



 iter  1
=====Validation=====
correlation : 0.0010953438572362591
max correlation index : 24
=====K-shot=====
k-shot correlation : 0.08385955932687596
k-shot max correlation index : 17
=====Test=====
age corr : 0.039569435674953135


 iter  2
=====Validation=====
correlation : -0.003146844492891949
max correlation index : 19
=====K-shot=====
k-shot correlation : -0.08028331011894053
k-shot max correlation index : 30
=====Test=====
age corr : 0.03247967623793676


 iter  3
=====Validation=====
correlation : -0.011244915406575266
max correlation index : 6
=====K-shot=====
k-shot correlation : 0.01109318715633381
k-shot max correlation index : 16
=====Test=====
age corr : -0.00814751940654704


 iter  4
=====Validation=====
correlation : 0.007083446732787483
max correlation index : 8
=====K-shot=====
k-shot correlation : -0.036572697718696444
k-shot max correlation index : 21
=====Test=====
age corr : 0.015869814085276403


 iter  5
=====Validation=====
correlation : -0.0038136490274763

In [65]:
basic_dnn_exp('f_lang' ,f_lang, 10, 10)
basic_dnn_exp('f_lang' ,f_lang, 30, 10)
basic_dnn_exp('f_lang' ,f_lang, 50, 10)



 iter  1
=====Validation=====
correlation : 0.04726168071764896
max correlation index : 9
=====K-shot=====
k-shot correlation : -0.21368632449408367
k-shot max correlation index : 7
=====Test=====
age corr : -0.07113315222837462


 iter  2
=====Validation=====
correlation : 0.021491284899930928
max correlation index : 35
=====K-shot=====
k-shot correlation : -0.02118868005103291
k-shot max correlation index : 27
=====Test=====
age corr : -0.017561222846919557


 iter  3
=====Validation=====
correlation : -0.03368962536708499
max correlation index : 39
=====K-shot=====
k-shot correlation : -0.16733519577601477
k-shot max correlation index : 39
=====Test=====
age corr : 0.13906119264520192


 iter  4
=====Validation=====
correlation : -0.03182474791764834
max correlation index : 43
=====K-shot=====
k-shot correlation : -0.024803654896198152
k-shot max correlation index : 16
=====Test=====
age corr : 0.06919486564185921


 iter  5
=====Validation=====
correlation : 0.017140382977982493


In [66]:
basic_dnn_exp('f_motor' ,f_motor, 10, 10)
basic_dnn_exp('f_motor' ,f_motor, 30, 10)
basic_dnn_exp('f_motor' ,f_motor, 50, 10)



 iter  1
=====Validation=====
correlation : 0.004990197268991158
max correlation index : 49
=====K-shot=====
k-shot correlation : 0.020164386876028888
k-shot max correlation index : 24
=====Test=====
age corr : 0.06836231593222654


 iter  2
=====Validation=====
correlation : -0.008171862485279369
max correlation index : 38
=====K-shot=====
k-shot correlation : 0.10013971181991406
k-shot max correlation index : 4
=====Test=====
age corr : -0.16133859232264958


 iter  3
=====Validation=====
correlation : -0.013485970063902343
max correlation index : 23
=====K-shot=====
k-shot correlation : -0.011934481491636048
k-shot max correlation index : 18
=====Test=====
age corr : 0.035779484060679004


 iter  4
=====Validation=====
correlation : 0.03190604043274143
max correlation index : 13
=====K-shot=====
k-shot correlation : -0.062280613901461834
k-shot max correlation index : 27
=====Test=====
age corr : -0.0231524016117984


 iter  5
=====Validation=====
correlation : 0.03474152338435613

In [67]:
basic_dnn_exp('f_rs' ,f_rs, 10, 10)
basic_dnn_exp('f_rs' ,f_rs, 30, 10)
basic_dnn_exp('f_rs' ,f_rs, 50, 10)



 iter  1
=====Validation=====
correlation : 0.008914864767609474
max correlation index : 3
=====K-shot=====
k-shot correlation : -0.05259980284205505
k-shot max correlation index : 34
=====Test=====
age corr : -0.08753790592441152


 iter  2
=====Validation=====
correlation : 0.006275274719216686
max correlation index : 34
=====K-shot=====
k-shot correlation : -0.17172605341897879
k-shot max correlation index : 21
=====Test=====
age corr : 0.12506442332099607


 iter  3
=====Validation=====
correlation : -0.04764214426297702
max correlation index : 6
=====K-shot=====
k-shot correlation : -0.22958223237861944
k-shot max correlation index : 43
=====Test=====
age corr : -0.05085588944379012


 iter  4
=====Validation=====
correlation : 0.036014169169366346
max correlation index : 15
=====K-shot=====
k-shot correlation : 0.19950876826189604
k-shot max correlation index : 20
=====Test=====
age corr : -0.008839126691470913


 iter  5
=====Validation=====
correlation : -0.012165969792504882

In [68]:
basic_dnn_exp('f_social' ,f_social, 10, 10)
basic_dnn_exp('f_social' ,f_social, 30, 10)
basic_dnn_exp('f_social' ,f_social, 50, 10)



 iter  1
=====Validation=====
correlation : -0.01961179512993733
max correlation index : 14
=====K-shot=====
k-shot correlation : -0.03191089571852277
k-shot max correlation index : 22
=====Test=====
age corr : -0.049302890578475


 iter  2
=====Validation=====
correlation : 0.019652747637976385
max correlation index : 49
=====K-shot=====
k-shot correlation : -0.1278843997742853
k-shot max correlation index : 55
=====Test=====
age corr : -0.08820283551299452


 iter  3
=====Validation=====
correlation : 0.0077892770328907325
max correlation index : 0
=====K-shot=====
k-shot correlation : 0.039225145318062364
k-shot max correlation index : 55
=====Test=====
age corr : -0.0018618592504857587


 iter  4
=====Validation=====
correlation : -0.03221102592216939
max correlation index : 28
=====K-shot=====
k-shot correlation : -0.07100169131435186
k-shot max correlation index : 22
=====Test=====
age corr : -0.025280269050971003


 iter  5
=====Validation=====
correlation : 0.0272510174701776

In [69]:
basic_dnn_exp('f_wm' ,f_wm, 10, 10)
basic_dnn_exp('f_wm' ,f_wm, 30, 10)
basic_dnn_exp('f_wm' ,f_wm, 50, 10)



 iter  1
=====Validation=====
correlation : -0.004046215967870448
max correlation index : 34
=====K-shot=====
k-shot correlation : -0.021868362929933053
k-shot max correlation index : 17
=====Test=====
age corr : 0.01970632271282741


 iter  2
=====Validation=====
correlation : 0.030641007104136583
max correlation index : 15
=====K-shot=====
k-shot correlation : -0.23582086496403204
k-shot max correlation index : 48
=====Test=====
age corr : 0.11515365042543317


 iter  3
=====Validation=====
correlation : -0.019500369368838515
max correlation index : 22
=====K-shot=====
k-shot correlation : -0.1588914037658027
k-shot max correlation index : 32
=====Test=====
age corr : -0.08596381365059377


 iter  4
=====Validation=====
correlation : -0.017815950779183225
max correlation index : 38
=====K-shot=====
k-shot correlation : -0.19107024297661154
k-shot max correlation index : 20
=====Test=====
age corr : -0.0915182378535961


 iter  5
=====Validation=====
correlation : -0.025681225238573

### Advanced Fine-Tuning

**sMRI**

In [70]:
advanced_finetune('cort_thick' ,cort_thick, 10, 10)
advanced_finetune('cort_thick' ,cort_thick, 30, 10)
advanced_finetune('cort_thick' ,cort_thick, 50, 10)



 iter  1
(121, 134) (121, 58)
===============Validation===============
correlation : 0.0103
max correlation index : 26
===============K-shot===============
k-shot correlation : -0.1478
k-shot max correlation index : 19
max corr pheno index 19
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.099 




 iter  2
(121, 134) (121, 58)
===============Validation===============
correlation : -0.0136
max correlation index : 30
===============K-shot===============
k-shot correlation : 0.1235
k-shot max correlation index : 38
max corr pheno index 38
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.1169 




 iter  3
(121, 134) (121, 58)
===============Validation===============
correlation : -0.0133
max correlation index : 54
===============K-shot===============
k-shot correlation : 0.0286
k-shot max correlation index : 13
max corr pheno index 13
=====

In [71]:
advanced_finetune('cort_area' ,cort_area, 10, 10)
advanced_finetune('cort_area' ,cort_area, 30, 10)
advanced_finetune('cort_area' ,cort_area, 50, 10)



 iter  1
(121, 106) (121, 58)
===============Validation===============
correlation : -0.0162
max correlation index : 25
===============K-shot===============
k-shot correlation : -0.0849
k-shot max correlation index : 19
max corr pheno index 19
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.1101 




 iter  2
(121, 106) (121, 58)
===============Validation===============
correlation : -0.0008
max correlation index : 19
===============K-shot===============
k-shot correlation : -0.0654
k-shot max correlation index : 38
max corr pheno index 38
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0619 




 iter  3
(121, 106) (121, 58)
===============Validation===============
correlation : 0.0053
max correlation index : 37
===============K-shot===============
k-shot correlation : 0.0302
k-shot max correlation index : 20
max corr pheno index 20
===

In [72]:
advanced_finetune('cort_vol' ,cort_vol, 10, 10)
advanced_finetune('cort_vol' ,cort_vol, 30, 10)
advanced_finetune('cort_vol' ,cort_vol, 50, 10)



 iter  1
(121, 119) (121, 58)
===============Validation===============
correlation : 0.0255
max correlation index : 42
===============K-shot===============
k-shot correlation : -0.2021
k-shot max correlation index : 51
max corr pheno index 51
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.0195 




 iter  2
(121, 119) (121, 58)
===============Validation===============
correlation : 0.0077
max correlation index : 57
===============K-shot===============
k-shot correlation : -0.1166
k-shot max correlation index : 49
max corr pheno index 49
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.1769 




 iter  3
(121, 119) (121, 58)
===============Validation===============
correlation : -0.0159
max correlation index : 36
===============K-shot===============
k-shot correlation : 0.0362
k-shot max correlation index : 22
max corr pheno index 22
=====

**TBSS**

In [73]:
advanced_finetune('tbss_ad' ,tbss_ad, 10, 10)
advanced_finetune('tbss_ad' ,tbss_ad, 30, 10)
advanced_finetune('tbss_ad' ,tbss_ad, 50, 10)



 iter  1
(121, 453) (121, 58)
===============Validation===============
correlation : 0.0145
max correlation index : 8
===============K-shot===============
k-shot correlation : -0.081
k-shot max correlation index : 46
max corr pheno index 46
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.157 




 iter  2
(121, 453) (121, 58)
===============Validation===============
correlation : -0.0327
max correlation index : 31
===============K-shot===============
k-shot correlation : -0.0822
k-shot max correlation index : 27
max corr pheno index 27
===============K-shot fine tuning validation===============
age corr nan
===============Test===============
age corr : -0.0327 




 iter  3
(121, 453) (121, 58)
===============Validation===============
correlation : -0.0198
max correlation index : 30
===============K-shot===============
k-shot correlation : 0.0081
k-shot max correlation index : 28
max corr pheno index 28
=======

In [74]:
advanced_finetune('tbss_fa' ,tbss_fa, 10, 10)
advanced_finetune('tbss_fa' ,tbss_fa, 30, 10)
advanced_finetune('tbss_fa' ,tbss_fa, 50, 10)



 iter  1
(121, 462) (121, 58)
===============Validation===============
correlation : 0.0352
max correlation index : 56
===============K-shot===============
k-shot correlation : -0.0692
k-shot max correlation index : 50
max corr pheno index 50
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.0802 




 iter  2
(121, 462) (121, 58)
===============Validation===============
correlation : 0.0016
max correlation index : 3
===============K-shot===============
k-shot correlation : 0.0588
k-shot max correlation index : 39
max corr pheno index 39
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.1679 




 iter  3
(121, 462) (121, 58)
===============Validation===============
correlation : 0.0078
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.0337
k-shot max correlation index : 23
max corr pheno index 23
=======

In [75]:
advanced_finetune('tbss_icvf' ,tbss_icvf, 10, 10)
advanced_finetune('tbss_icvf' ,tbss_icvf, 30, 10)
advanced_finetune('tbss_icvf' ,tbss_icvf, 50, 10)



 iter  1
(121, 438) (121, 58)
===============Validation===============
correlation : -0.0045
max correlation index : 38
===============K-shot===============
k-shot correlation : -0.0722
k-shot max correlation index : 27
max corr pheno index 27
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.1605 




 iter  2
(121, 438) (121, 58)
===============Validation===============
correlation : -0.0102
max correlation index : 23
===============K-shot===============
k-shot correlation : 0.0268
k-shot max correlation index : 28
max corr pheno index 28
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.01 




 iter  3
(121, 438) (121, 58)
===============Validation===============
correlation : 0.037
max correlation index : 47
===============K-shot===============
k-shot correlation : -0.0817
k-shot max correlation index : 51
max corr pheno index 51
========

In [76]:
advanced_finetune('tbss_isovf' ,tbss_isovf, 10, 10)
advanced_finetune('tbss_isovf' ,tbss_isovf, 30, 10)
advanced_finetune('tbss_isovf' ,tbss_isovf, 50, 10)



 iter  1
(121, 475) (121, 58)
===============Validation===============
correlation : -0.0299
max correlation index : 47
===============K-shot===============
k-shot correlation : -0.0942
k-shot max correlation index : 51
max corr pheno index 51
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.0925 




 iter  2
(121, 475) (121, 58)
===============Validation===============
correlation : 0.0013
max correlation index : 12
===============K-shot===============
k-shot correlation : 0.1013
k-shot max correlation index : 52
max corr pheno index 52
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.1302 




 iter  3
(121, 475) (121, 58)
===============Validation===============
correlation : -0.0155
max correlation index : 21
===============K-shot===============
k-shot correlation : -0.0174
k-shot max correlation index : 36
max corr pheno index 36
==

In [77]:
advanced_finetune('tbss_md' ,tbss_md, 10, 10)
advanced_finetune('tbss_md' ,tbss_md, 30, 10)
advanced_finetune('tbss_md' ,tbss_md, 50, 10)



 iter  1
(121, 415) (121, 58)
===============Validation===============
correlation : 0.009
max correlation index : 35
===============K-shot===============
k-shot correlation : 0.185
k-shot max correlation index : 15
max corr pheno index 15
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.0075 




 iter  2
(121, 415) (121, 58)
===============Validation===============
correlation : -0.0107
max correlation index : 21
===============K-shot===============
k-shot correlation : 0.1013
k-shot max correlation index : 9
max corr pheno index 9
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0952 




 iter  3
(121, 415) (121, 58)
===============Validation===============
correlation : -0.0088
max correlation index : 4
===============K-shot===============
k-shot correlation : 0.2247
k-shot max correlation index : 37
max corr pheno index 37
============

In [78]:
advanced_finetune('tbss_od' ,tbss_od, 10, 10)
advanced_finetune('tbss_od' ,tbss_od, 30, 10)
advanced_finetune('tbss_od' ,tbss_od, 50, 10)



 iter  1
(121, 514) (121, 58)
===============Validation===============
correlation : -0.0032
max correlation index : 53
===============K-shot===============
k-shot correlation : -0.0287
k-shot max correlation index : 47
max corr pheno index 47
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.0428 




 iter  2
(121, 514) (121, 58)
===============Validation===============
correlation : -0.0665
max correlation index : 28
===============K-shot===============
k-shot correlation : 0.0646
k-shot max correlation index : 9
max corr pheno index 9
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.0159 




 iter  3
(121, 514) (121, 58)
===============Validation===============
correlation : -0.0277
max correlation index : 14
===============K-shot===============
k-shot correlation : 0.0209
k-shot max correlation index : 33
max corr pheno index 33
======

In [79]:
advanced_finetune('tbss_rd' ,tbss_rd, 10, 10)
advanced_finetune('tbss_rd' ,tbss_rd, 30, 10)
advanced_finetune('tbss_rd' ,tbss_rd, 50, 10)



 iter  1
(121, 455) (121, 58)
===============Validation===============
correlation : 0.0001
max correlation index : 17
===============K-shot===============
k-shot correlation : -0.1084
k-shot max correlation index : 10
max corr pheno index 10
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0308 




 iter  2
(121, 455) (121, 58)
===============Validation===============
correlation : -0.0361
max correlation index : 17
===============K-shot===============
k-shot correlation : 0.1121
k-shot max correlation index : 2
max corr pheno index 2
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.025 




 iter  3
(121, 455) (121, 58)
===============Validation===============
correlation : -0.0187
max correlation index : 47
===============K-shot===============
k-shot correlation : 0.1184
k-shot max correlation index : 10
max corr pheno index 10
========

**Tractography**

In [80]:
advanced_finetune('tractography_ad' ,tractography_ad, 10, 10)
advanced_finetune('tractography_ad' ,tractography_ad, 30, 10)
advanced_finetune('tractography_ad' ,tractography_ad, 50, 10)



 iter  1
(121, 504) (121, 58)
===============Validation===============
correlation : 0.0072
max correlation index : 31
===============K-shot===============
k-shot correlation : -0.0016
k-shot max correlation index : 7
max corr pheno index 7
===============K-shot fine tuning validation===============
age corr nan
===============Test===============
age corr : -0.0887 




 iter  2
(121, 504) (121, 58)
===============Validation===============
correlation : 0.0069
max correlation index : 43
===============K-shot===============
k-shot correlation : -0.1313
k-shot max correlation index : 17
max corr pheno index 17
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0277 




 iter  3
(121, 504) (121, 58)
===============Validation===============
correlation : 0.0503
max correlation index : 8
===============K-shot===============
k-shot correlation : -0.059
k-shot max correlation index : 38
max corr pheno index 38
=========

In [81]:
advanced_finetune('tractography_fa' ,tractography_fa, 10, 10)
advanced_finetune('tractography_fa' ,tractography_fa, 30, 10)
advanced_finetune('tractography_fa' ,tractography_fa, 50, 10)



 iter  1
(121, 503) (121, 58)
===============Validation===============
correlation : 0.0088
max correlation index : 18
===============K-shot===============
k-shot correlation : 0.148
k-shot max correlation index : 7
max corr pheno index 7
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.037 




 iter  2
(121, 503) (121, 58)
===============Validation===============
correlation : -0.0415
max correlation index : 42
===============K-shot===============
k-shot correlation : -0.0299
k-shot max correlation index : 45
max corr pheno index 45
===============K-shot fine tuning validation===============
age corr nan
===============Test===============
age corr : 0.0096 




 iter  3
(121, 503) (121, 58)
===============Validation===============
correlation : -0.034
max correlation index : 39
===============K-shot===============
k-shot correlation : -0.0829
k-shot max correlation index : 30
max corr pheno index 30
===========

In [82]:
advanced_finetune('tractography_icvf' ,tractography_icvf, 10, 10)
advanced_finetune('tractography_icvf' ,tractography_icvf, 30, 10)
advanced_finetune('tractography_icvf' ,tractography_icvf, 50, 10)



 iter  1
(121, 497) (121, 58)
===============Validation===============
correlation : -0.003
max correlation index : 0
===============K-shot===============
k-shot correlation : -0.0666
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.0368 




 iter  2
(121, 497) (121, 58)
===============Validation===============
correlation : 0.0345
max correlation index : 53
===============K-shot===============
k-shot correlation : -0.0905
k-shot max correlation index : 14
max corr pheno index 14
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.1314 




 iter  3
(121, 497) (121, 58)
===============Validation===============
correlation : 0.0112
max correlation index : 57
===============K-shot===============
k-shot correlation : -0.0207
k-shot max correlation index : 57
max corr pheno index 57
=======

In [83]:
advanced_finetune('tractography_isovf' ,tractography_isovf, 10, 10)
advanced_finetune('tractography_isovf' ,tractography_isovf, 30, 10)
advanced_finetune('tractography_isovf' ,tractography_isovf, 50, 10)



 iter  1
(121, 476) (121, 58)
===============Validation===============
correlation : -0.0032
max correlation index : 48
===============K-shot===============
k-shot correlation : -0.1906
k-shot max correlation index : 28
max corr pheno index 28
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.0552 




 iter  2
(121, 476) (121, 58)
===============Validation===============
correlation : -0.0021
max correlation index : 13
===============K-shot===============
k-shot correlation : -0.0626
k-shot max correlation index : 28
max corr pheno index 28
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.0764 




 iter  3
(121, 476) (121, 58)
===============Validation===============
correlation : -0.0091
max correlation index : 28
===============K-shot===============
k-shot correlation : -0.0792
k-shot max correlation index : 0
max corr pheno index 0
====

In [84]:
advanced_finetune('tractography_md' ,tractography_md, 10, 10)
advanced_finetune('tractography_md' ,tractography_md, 30, 10)
advanced_finetune('tractography_md' ,tractography_md, 50, 10)



 iter  1
(121, 500) (121, 58)
===============Validation===============
correlation : -0.041
max correlation index : 28
===============K-shot===============
k-shot correlation : 0.0391
k-shot max correlation index : 47
max corr pheno index 47
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0224 




 iter  2
(121, 500) (121, 58)
===============Validation===============
correlation : 0.0144
max correlation index : 16
===============K-shot===============
k-shot correlation : 0.1214
k-shot max correlation index : 11
max corr pheno index 11
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.1569 




 iter  3
(121, 500) (121, 58)
===============Validation===============
correlation : -0.0066
max correlation index : 47
===============K-shot===============
k-shot correlation : 0.0221
k-shot max correlation index : 14
max corr pheno index 14
========

In [85]:
advanced_finetune('tractography_od' ,tractography_od, 10, 10)
advanced_finetune('tractography_od' ,tractography_od, 30, 10)
advanced_finetune('tractography_od' ,tractography_od, 50, 10)



 iter  1
(121, 504) (121, 58)
===============Validation===============
correlation : -0.0226
max correlation index : 26
===============K-shot===============
k-shot correlation : 0.0589
k-shot max correlation index : 13
max corr pheno index 13
===============K-shot fine tuning validation===============
age corr nan
===============Test===============
age corr : -0.0084 




 iter  2
(121, 504) (121, 58)
===============Validation===============
correlation : 0.0053
max correlation index : 57
===============K-shot===============
k-shot correlation : 0.0598
k-shot max correlation index : 24
max corr pheno index 24
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0119 




 iter  3
(121, 504) (121, 58)
===============Validation===============
correlation : -0.0061
max correlation index : 38
===============K-shot===============
k-shot correlation : -0.0434
k-shot max correlation index : 17
max corr pheno index 17
=====

In [86]:
advanced_finetune('tractography_rd' ,tractography_rd, 10, 10)
advanced_finetune('tractography_rd' ,tractography_rd, 30, 10)
advanced_finetune('tractography_rd' ,tractography_rd, 50, 10)



 iter  1
(121, 498) (121, 58)
===============Validation===============
correlation : -0.0184
max correlation index : 26
===============K-shot===============
k-shot correlation : 0.1743
k-shot max correlation index : 19
max corr pheno index 19
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0982 




 iter  2
(121, 498) (121, 58)
===============Validation===============
correlation : -0.0439
max correlation index : 32
===============K-shot===============
k-shot correlation : -0.0848
k-shot max correlation index : 42
max corr pheno index 42
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.0472 




 iter  3
(121, 498) (121, 58)
===============Validation===============
correlation : 0.0294
max correlation index : 46
===============K-shot===============
k-shot correlation : 0.1136
k-shot max correlation index : 39
max corr pheno index 39
=====

In [87]:
advanced_finetune('tractography_fss' ,tractography_fss, 10, 10)
advanced_finetune('tractography_fss' ,tractography_fss, 30, 10)
advanced_finetune('tractography_fss' ,tractography_fss, 50, 10)



 iter  1
(121, 521) (121, 58)
===============Validation===============
correlation : -0.0101
max correlation index : 29
===============K-shot===============
k-shot correlation : 0.02
k-shot max correlation index : 41
max corr pheno index 41
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.1511 




 iter  2
(121, 521) (121, 58)
===============Validation===============
correlation : 0.0277
max correlation index : 25
===============K-shot===============
k-shot correlation : 0.0012
k-shot max correlation index : 32
max corr pheno index 32
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.0114 




 iter  3
(121, 521) (121, 58)
===============Validation===============
correlation : -0.0157
max correlation index : 17
===============K-shot===============
k-shot correlation : -0.0272
k-shot max correlation index : 14
max corr pheno index 14
=====

In [88]:
advanced_finetune('tractography_fssl' ,tractography_fssl, 10, 10)
advanced_finetune('tractography_fssl' ,tractography_fssl, 30, 10)
advanced_finetune('tractography_fssl' ,tractography_fssl, 50, 10)



 iter  1
(121, 456) (121, 58)
===============Validation===============
correlation : 0.0454
max correlation index : 9
===============K-shot===============
k-shot correlation : 0.0174
k-shot max correlation index : 45
max corr pheno index 45
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.1504 




 iter  2
(121, 456) (121, 58)
===============Validation===============
correlation : -0.0581
max correlation index : 32
===============K-shot===============
k-shot correlation : 0.0618
k-shot max correlation index : 55
max corr pheno index 55
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.1181 




 iter  3
(121, 456) (121, 58)
===============Validation===============
correlation : -0.0151
max correlation index : 23
===============K-shot===============
k-shot correlation : -0.0028
k-shot max correlation index : 3
max corr pheno index 3
=======

**fMRI**

In [89]:
advanced_finetune('f_gamb' ,f_gamb, 10, 10)
advanced_finetune('f_gamb' ,f_gamb, 30, 10)
advanced_finetune('f_gamb' ,f_gamb, 50, 10)



 iter  1
(121, 386) (121, 58)
===============Validation===============
correlation : 0.0085
max correlation index : 32
===============K-shot===============
k-shot correlation : -0.1065
k-shot max correlation index : 42
max corr pheno index 42
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0367 




 iter  2
(121, 386) (121, 58)
===============Validation===============
correlation : 0.0058
max correlation index : 7
===============K-shot===============
k-shot correlation : -0.0128
k-shot max correlation index : 34
max corr pheno index 34
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.0061 




 iter  3
(121, 386) (121, 58)
===============Validation===============
correlation : 0.0218
max correlation index : 13
===============K-shot===============
k-shot correlation : -0.0308
k-shot max correlation index : 20
max corr pheno index 20
=======

In [90]:
advanced_finetune('f_lang' ,f_lang, 10, 10)
advanced_finetune('f_lang' ,f_lang, 30, 10)
advanced_finetune('f_lang' ,f_lang, 50, 10)



 iter  1
(121, 371) (121, 58)
===============Validation===============
correlation : -0.038
max correlation index : 39
===============K-shot===============
k-shot correlation : -0.05
k-shot max correlation index : 30
max corr pheno index 30
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.0813 




 iter  2
(121, 371) (121, 58)
===============Validation===============
correlation : -0.0181
max correlation index : 10
===============K-shot===============
k-shot correlation : -0.0785
k-shot max correlation index : 22
max corr pheno index 22
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : -0.0002 




 iter  3
(121, 371) (121, 58)
===============Validation===============
correlation : -0.0518
max correlation index : 39
===============K-shot===============
k-shot correlation : 0.1286
k-shot max correlation index : 16
max corr pheno index 16
======

In [91]:
advanced_finetune('f_motor' ,f_motor, 10, 10)
advanced_finetune('f_motor' ,f_motor, 30, 10)
advanced_finetune('f_motor' ,f_motor, 50, 10)



 iter  1
(121, 386) (121, 58)
===============Validation===============
correlation : 0.0034
max correlation index : 34
===============K-shot===============
k-shot correlation : -0.062
k-shot max correlation index : 38
max corr pheno index 38
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.1282 




 iter  2
(121, 386) (121, 58)
===============Validation===============
correlation : -0.0003
max correlation index : 10
===============K-shot===============
k-shot correlation : -0.1082
k-shot max correlation index : 21
max corr pheno index 21
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.1398 




 iter  3
(121, 386) (121, 58)
===============Validation===============
correlation : -0.023
max correlation index : 16
===============K-shot===============
k-shot correlation : -0.1352
k-shot max correlation index : 23
max corr pheno index 23
=======

In [92]:
advanced_finetune('f_rs' ,f_rs, 10, 10)
advanced_finetune('f_rs' ,f_rs, 30, 10)
advanced_finetune('f_rs' ,f_rs, 50, 10)



 iter  1
(121, 312) (121, 58)
===============Validation===============
correlation : 0.0028
max correlation index : 6
===============K-shot===============
k-shot correlation : 0.0269
k-shot max correlation index : 37
max corr pheno index 37
===============K-shot fine tuning validation===============
age corr 1.0
===============Test===============
age corr : 0.1757 




 iter  2
(121, 312) (121, 58)
===============Validation===============
correlation : -0.0166
max correlation index : 30
===============K-shot===============
k-shot correlation : -0.1328
k-shot max correlation index : 54
max corr pheno index 54
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.1161 




 iter  3
(121, 312) (121, 58)
===============Validation===============
correlation : -0.0026
max correlation index : 21
===============K-shot===============
k-shot correlation : 0.017
k-shot max correlation index : 13
max corr pheno index 13
========

In [93]:
advanced_finetune('f_social' ,f_social, 10, 10)
advanced_finetune('f_social' ,f_social, 30, 10)
advanced_finetune('f_social' ,f_social, 50, 10)



 iter  1
(121, 352) (121, 58)
===============Validation===============
correlation : 0.051
max correlation index : 39
===============K-shot===============
k-shot correlation : -0.195
k-shot max correlation index : 11
max corr pheno index 11
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.124 




 iter  2
(121, 352) (121, 58)
===============Validation===============
correlation : -0.0025
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.1431
k-shot max correlation index : 22
max corr pheno index 22
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.0294 




 iter  3
(121, 352) (121, 58)
===============Validation===============
correlation : -0.0119
max correlation index : 57
===============K-shot===============
k-shot correlation : -0.1812
k-shot max correlation index : 55
max corr pheno index 55
=====

In [94]:
advanced_finetune('f_wm' ,f_wm, 10, 10)
advanced_finetune('f_wm' ,f_wm, 30, 10)
advanced_finetune('f_wm' ,f_wm, 50, 10)



 iter  1
(121, 339) (121, 58)
===============Validation===============
correlation : 0.0005
max correlation index : 14
===============K-shot===============
k-shot correlation : -0.0417
k-shot max correlation index : 5
max corr pheno index 5
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : -0.1007 




 iter  2
(121, 339) (121, 58)
===============Validation===============
correlation : -0.0096
max correlation index : 30
===============K-shot===============
k-shot correlation : -0.0305
k-shot max correlation index : 45
max corr pheno index 45
===============K-shot fine tuning validation===============
age corr -1.0
===============Test===============
age corr : 0.0347 




 iter  3
(121, 339) (121, 58)
===============Validation===============
correlation : 0.0093
max correlation index : 31
===============K-shot===============
k-shot correlation : -0.0601
k-shot max correlation index : 39
max corr pheno index 39
=====

### Advanced - Stacking

**sMRI**

In [95]:
advanced_stacking('cort_thick' ,cort_thick, 10, 10)
advanced_stacking('cort_thick' ,cort_thick, 30, 10)
advanced_stacking('cort_thick' ,cort_thick, 50, 10)



 iter  1
=====Validation=====
correlation : -0.007360729386759487
max correlation index : 24
=====Test=====
age corr : -0.1345


 iter  2
=====Validation=====
correlation : -0.026548669625744377
max correlation index : 13
=====Test=====
age corr : -0.0134


 iter  3
=====Validation=====
correlation : 0.0455022940247535
max correlation index : 52
=====Test=====
age corr : -0.0791


 iter  4
=====Validation=====
correlation : -0.011633067925490584
max correlation index : 3
=====Test=====
age corr : -0.0181


 iter  5
=====Validation=====
correlation : -0.026743759592690437
max correlation index : 22
=====Test=====
age corr : -0.0489


 iter  6
=====Validation=====
correlation : -0.019503045584633817
max correlation index : 28
=====Test=====
age corr : -0.0105


 iter  7
=====Validation=====
correlation : 0.0076644844207512716
max correlation index : 23
=====Test=====
age corr : 0.0238


 iter  8
=====Validation=====
correlation : 0.041696213200983254
max correlation index : 56
=====Tes

In [96]:
advanced_stacking('cort_area' ,cort_area, 10, 10)
advanced_stacking('cort_area' ,cort_area, 30, 10)
advanced_stacking('cort_area' ,cort_area, 50, 10)



 iter  1
=====Validation=====
correlation : 0.025769155480799284
max correlation index : 26
=====Test=====
age corr : 0.0799


 iter  2
=====Validation=====
correlation : -0.015300579183405626
max correlation index : 2
=====Test=====
age corr : -0.0712


 iter  3
=====Validation=====
correlation : 0.020019592447787057
max correlation index : 40
=====Test=====
age corr : -0.0063


 iter  4
=====Validation=====
correlation : -0.04299067566774096
max correlation index : 37
=====Test=====
age corr : -0.0197


 iter  5
=====Validation=====
correlation : -0.03879754236542703
max correlation index : 8
=====Test=====
age corr : -0.0331


 iter  6
=====Validation=====
correlation : 0.00975563125290266
max correlation index : 46
=====Test=====
age corr : 0.1173


 iter  7
=====Validation=====
correlation : -0.019960524601028855
max correlation index : 9
=====Test=====
age corr : 0.0089


 iter  8
=====Validation=====
correlation : 0.02137960211470361
max correlation index : 51
=====Test=====
a

In [97]:
advanced_stacking('cort_vol' ,cort_vol, 10, 10)
advanced_stacking('cort_vol' ,cort_vol, 30, 10)
advanced_stacking('cort_vol' ,cort_vol, 50, 10)



 iter  1
=====Validation=====
correlation : -0.0009737190966282154
max correlation index : 14
=====Test=====
age corr : 0.0676


 iter  2
=====Validation=====
correlation : -0.02919613643741073
max correlation index : 18
=====Test=====
age corr : 0.0372


 iter  3
=====Validation=====
correlation : 0.029315180441107556
max correlation index : 40
=====Test=====
age corr : 0.0098


 iter  4
=====Validation=====
correlation : -0.004136267258394808
max correlation index : 0
=====Test=====
age corr : -0.0031


 iter  5
=====Validation=====
correlation : 0.025032060160667037
max correlation index : 23
=====Test=====
age corr : -0.1371


 iter  6
=====Validation=====
correlation : -0.004514087931780557
max correlation index : 20
=====Test=====
age corr : 0.2520


 iter  7
=====Validation=====
correlation : 0.0027013749978935016
max correlation index : 23
=====Test=====
age corr : 0.0229


 iter  8
=====Validation=====
correlation : 0.008696734420394115
max correlation index : 31
=====Test==

**TBSS**

In [98]:
advanced_stacking('tbss_ad' ,tbss_ad, 10, 10)
advanced_stacking('tbss_ad' ,tbss_ad, 30, 10)
advanced_stacking('tbss_ad' ,tbss_ad, 50, 10)



 iter  1
=====Validation=====
correlation : -0.0013177143466901124
max correlation index : 56
=====Test=====
age corr : 0.1172


 iter  2
=====Validation=====
correlation : 0.03324505534389246
max correlation index : 47
=====Test=====
age corr : -0.0857


 iter  3
=====Validation=====
correlation : 0.03826323060536223
max correlation index : 4
=====Test=====
age corr : -0.0394


 iter  4
=====Validation=====
correlation : -0.00905309796270697
max correlation index : 40
=====Test=====
age corr : -0.0397


 iter  5
=====Validation=====
correlation : -0.0037042639226712677
max correlation index : 3
=====Test=====
age corr : -0.0142


 iter  6
=====Validation=====
correlation : -0.015428390590337692
max correlation index : 31
=====Test=====
age corr : -0.0768


 iter  7
=====Validation=====
correlation : 0.013774756988487524
max correlation index : 51
=====Test=====
age corr : -0.0579


 iter  8
=====Validation=====
correlation : 0.0075896864145250575
max correlation index : 45
=====Test

In [99]:
advanced_stacking('tbss_fa' ,tbss_fa, 10, 10)
advanced_stacking('tbss_fa' ,tbss_fa, 30, 10)
advanced_stacking('tbss_fa' ,tbss_fa, 50, 10)



 iter  1
=====Validation=====
correlation : 0.022060247089710792
max correlation index : 6
=====Test=====
age corr : 0.0128


 iter  2
=====Validation=====
correlation : -0.007806934234085596
max correlation index : 20
=====Test=====
age corr : -0.0590


 iter  3
=====Validation=====
correlation : -0.0012661985233494114
max correlation index : 28
=====Test=====
age corr : -0.1409


 iter  4
=====Validation=====
correlation : -0.015083145353686645
max correlation index : 6
=====Test=====
age corr : -0.0657


 iter  5
=====Validation=====
correlation : 0.024400585312590324
max correlation index : 55
=====Test=====
age corr : 0.0485


 iter  6
=====Validation=====
correlation : nan
max correlation index : 20
=====Test=====
age corr : 0.0592


 iter  7
=====Validation=====
correlation : 0.0018852823398565414
max correlation index : 7
=====Test=====
age corr : -0.1106


 iter  8
=====Validation=====
correlation : -0.011010623679435566
max correlation index : 46
=====Test=====
age corr : 0

In [100]:
advanced_stacking('tbss_icvf' ,tbss_icvf, 10, 10)
advanced_stacking('tbss_icvf' ,tbss_icvf, 30, 10)
advanced_stacking('tbss_icvf' ,tbss_icvf, 50, 10)



 iter  1
=====Validation=====
correlation : 0.032003727166799625
max correlation index : 52
=====Test=====
age corr : -0.2762


 iter  2
=====Validation=====
correlation : -0.039809298024008996
max correlation index : 8
=====Test=====
age corr : -0.1089


 iter  3
=====Validation=====
correlation : -0.031636972908056664
max correlation index : 41
=====Test=====
age corr : 0.0613


 iter  4
=====Validation=====
correlation : -0.00016988131128677444
max correlation index : 22
=====Test=====
age corr : 0.0663


 iter  5
=====Validation=====
correlation : 0.007585990612242669
max correlation index : 15
=====Test=====
age corr : -0.1929


 iter  6
=====Validation=====
correlation : 0.014835480125691058
max correlation index : 52
=====Test=====
age corr : -0.1046


 iter  7
=====Validation=====
correlation : 0.03684569582883197
max correlation index : 16
=====Test=====
age corr : 0.0095


 iter  8
=====Validation=====
correlation : -0.002875352589492388
max correlation index : 6
=====Test=

In [101]:
advanced_stacking('tbss_isovf' ,tbss_isovf, 10, 10)
advanced_stacking('tbss_isovf' ,tbss_isovf, 30, 10)
advanced_stacking('tbss_isovf' ,tbss_isovf, 50, 10)



 iter  1
=====Validation=====
correlation : 0.01664692021042405
max correlation index : 23
=====Test=====
age corr : -0.0617


 iter  2
=====Validation=====
correlation : 0.004805717706925907
max correlation index : 54
=====Test=====
age corr : 0.0254


 iter  3
=====Validation=====
correlation : -0.033575137273133646
max correlation index : 28
=====Test=====
age corr : 0.1259


 iter  4
=====Validation=====
correlation : 0.010284843317544208
max correlation index : 33
=====Test=====
age corr : -0.1467


 iter  5
=====Validation=====
correlation : -0.035661968944611815
max correlation index : 48
=====Test=====
age corr : -0.0567


 iter  6
=====Validation=====
correlation : 0.05435208941360961
max correlation index : 57
=====Test=====
age corr : 0.0277


 iter  7
=====Validation=====
correlation : -0.062290027519182785
max correlation index : 39
=====Test=====
age corr : 0.0415


 iter  8
=====Validation=====
correlation : -0.04136984107305901
max correlation index : 19
=====Test====

In [102]:
advanced_stacking('tbss_md' ,tbss_md, 10, 10)
advanced_stacking('tbss_md' ,tbss_md, 30, 10)
advanced_stacking('tbss_md' ,tbss_md, 50, 10)



 iter  1
=====Validation=====
correlation : 0.0067918605518167245
max correlation index : 6
=====Test=====
age corr : -0.0123


 iter  2
=====Validation=====
correlation : -0.00027170832451391797
max correlation index : 15
=====Test=====
age corr : 0.1094


 iter  3
=====Validation=====
correlation : -0.005006016831933851
max correlation index : 23
=====Test=====
age corr : 0.0102


 iter  4
=====Validation=====
correlation : 0.010007848571471946
max correlation index : 29
=====Test=====
age corr : -0.0969


 iter  5
=====Validation=====
correlation : 0.00792550635543182
max correlation index : 39
=====Test=====
age corr : -0.1276


 iter  6
=====Validation=====
correlation : 0.008487160798618925
max correlation index : 29
=====Test=====
age corr : 0.0348


 iter  7
=====Validation=====
correlation : 0.006321554778034951
max correlation index : 10
=====Test=====
age corr : 0.0282


 iter  8
=====Validation=====
correlation : -0.01682652352181481
max correlation index : 8
=====Test===

In [103]:
advanced_stacking('tbss_od' ,tbss_od, 10, 10)
advanced_stacking('tbss_od' ,tbss_od, 30, 10)
advanced_stacking('tbss_od' ,tbss_od, 50, 10)



 iter  1
=====Validation=====
correlation : 0.034619304595117
max correlation index : 5
=====Test=====
age corr : 0.0993


 iter  2
=====Validation=====
correlation : 0.024084947982612546
max correlation index : 35
=====Test=====
age corr : -0.0741


 iter  3
=====Validation=====
correlation : 0.019840369277780632
max correlation index : 38
=====Test=====
age corr : -0.0438


 iter  4
=====Validation=====
correlation : -0.00257967535947093
max correlation index : 37
=====Test=====
age corr : -0.0308


 iter  5
=====Validation=====
correlation : -0.00328125372440324
max correlation index : 40
=====Test=====
age corr : 0.1311


 iter  6
=====Validation=====
correlation : 0.009896599751452568
max correlation index : 55
=====Test=====
age corr : 0.0336


 iter  7
=====Validation=====
correlation : 0.02435469555813131
max correlation index : 5
=====Test=====
age corr : -0.0474


 iter  8
=====Validation=====
correlation : -0.009550187398128327
max correlation index : 19
=====Test=====
age

In [104]:
advanced_stacking('tbss_rd' ,tbss_rd, 10, 10)
advanced_stacking('tbss_rd' ,tbss_rd, 30, 10)
advanced_stacking('tbss_rd' ,tbss_rd, 50, 10)



 iter  1
=====Validation=====
correlation : 0.005667602965738362
max correlation index : 43
=====Test=====
age corr : 0.0705


 iter  2
=====Validation=====
correlation : 0.01315164058973567
max correlation index : 15
=====Test=====
age corr : 0.0196


 iter  3
=====Validation=====
correlation : 0.023837096155468793
max correlation index : 16
=====Test=====
age corr : -0.0254


 iter  4
=====Validation=====
correlation : -0.00923707157808293
max correlation index : 26
=====Test=====
age corr : 0.0703


 iter  5
=====Validation=====
correlation : 0.007794408709617707
max correlation index : 57
=====Test=====
age corr : -0.0166


 iter  6
=====Validation=====
correlation : -0.022753110148165095
max correlation index : 18
=====Test=====
age corr : -0.0163


 iter  7
=====Validation=====
correlation : 0.027724324179444683
max correlation index : 21
=====Test=====
age corr : -0.0431


 iter  8
=====Validation=====
correlation : -0.03566364409488688
max correlation index : 27
=====Test====

**Tractography**

In [105]:
advanced_stacking('tractography_ad' ,tractography_ad, 10, 10)
advanced_stacking('tractography_ad' ,tractography_ad, 30, 10)
advanced_stacking('tractography_ad' ,tractography_ad, 50, 10)



 iter  1
=====Validation=====
correlation : 0.006982947165772707
max correlation index : 51
=====Test=====
age corr : 0.1194


 iter  2
=====Validation=====
correlation : -0.010014387886306375
max correlation index : 41
=====Test=====
age corr : -0.1941


 iter  3
=====Validation=====
correlation : -0.016360561298017037
max correlation index : 36
=====Test=====
age corr : 0.0571


 iter  4
=====Validation=====
correlation : -0.01616290192122928
max correlation index : 31
=====Test=====
age corr : 0.0621


 iter  5
=====Validation=====
correlation : 0.035793209601990957
max correlation index : 28
=====Test=====
age corr : -0.1609


 iter  6
=====Validation=====
correlation : -0.013866507239868226
max correlation index : 17
=====Test=====
age corr : 0.0034


 iter  7
=====Validation=====
correlation : 0.011869288215060841
max correlation index : 2
=====Test=====
age corr : 0.0839


 iter  8
=====Validation=====
correlation : -0.012703975035481119
max correlation index : 13
=====Test===

In [106]:
advanced_stacking('tractography_fa' ,tractography_fa, 10, 10)
advanced_stacking('tractography_fa' ,tractography_fa, 30, 10)
advanced_stacking('tractography_fa' ,tractography_fa, 50, 10)



 iter  1
=====Validation=====
correlation : -0.009818346424228023
max correlation index : 8
=====Test=====
age corr : 0.0475


 iter  2
=====Validation=====
correlation : 0.004943368862976518
max correlation index : 2
=====Test=====
age corr : -0.0620


 iter  3
=====Validation=====
correlation : -0.010455703086137436
max correlation index : 10
=====Test=====
age corr : 0.0548


 iter  4
=====Validation=====
correlation : -0.03234203548126162
max correlation index : 26
=====Test=====
age corr : -0.1063


 iter  5
=====Validation=====
correlation : 0.008143645295459455
max correlation index : 27
=====Test=====
age corr : 0.0250


 iter  6
=====Validation=====
correlation : 0.005036399540993855
max correlation index : 52
=====Test=====
age corr : 0.0717


 iter  7
=====Validation=====
correlation : -0.02515332416782235
max correlation index : 35
=====Test=====
age corr : 0.0137


 iter  8
=====Validation=====
correlation : -0.00587207073260412
max correlation index : 40
=====Test=====


In [107]:
advanced_stacking('tractography_icvf' ,tractography_icvf, 10, 10)
advanced_stacking('tractography_icvf' ,tractography_icvf, 30, 10)
advanced_stacking('tractography_icvf' ,tractography_icvf, 50, 10)



 iter  1
=====Validation=====
correlation : 0.013669948959236665
max correlation index : 9
=====Test=====
age corr : -0.1432


 iter  2
=====Validation=====
correlation : 0.016357021694004416
max correlation index : 11
=====Test=====
age corr : 0.0098


 iter  3
=====Validation=====
correlation : 0.027392477809586768
max correlation index : 39
=====Test=====
age corr : -0.1036


 iter  4
=====Validation=====
correlation : 0.008199570995811027
max correlation index : 48
=====Test=====
age corr : -0.0527


 iter  5
=====Validation=====
correlation : -0.0024595819598012417
max correlation index : 42
=====Test=====
age corr : -0.0299


 iter  6
=====Validation=====
correlation : -0.009710433323986502
max correlation index : 45
=====Test=====
age corr : 0.0820


 iter  7
=====Validation=====
correlation : -0.0021385138958320876
max correlation index : 29
=====Test=====
age corr : 0.0093


 iter  8
=====Validation=====
correlation : -0.02986785086475903
max correlation index : 47
=====Test

In [108]:
advanced_stacking('tractography_isovf' ,tractography_isovf, 10, 10)
advanced_stacking('tractography_isovf' ,tractography_isovf, 30, 10)
advanced_stacking('tractography_isovf' ,tractography_isovf, 50, 10)



 iter  1
=====Validation=====
correlation : -0.03088398354894235
max correlation index : 0
=====Test=====
age corr : 0.0275


 iter  2
=====Validation=====
correlation : 0.014108302438840288
max correlation index : 22
=====Test=====
age corr : 0.0846


 iter  3
=====Validation=====
correlation : 0.03291635946345013
max correlation index : 56
=====Test=====
age corr : 0.1376


 iter  4
=====Validation=====
correlation : 0.005037823802227898
max correlation index : 26
=====Test=====
age corr : -0.0349


 iter  5
=====Validation=====
correlation : 0.027511657709107896
max correlation index : 39
=====Test=====
age corr : 0.0950


 iter  6
=====Validation=====
correlation : 0.016822327946280885
max correlation index : 1
=====Test=====
age corr : -0.1120


 iter  7
=====Validation=====
correlation : -0.03736178631512294
max correlation index : 32
=====Test=====
age corr : 0.0004


 iter  8
=====Validation=====
correlation : -0.011534522392261654
max correlation index : 47
=====Test=====
ag

In [109]:
advanced_stacking('tractography_md' ,tractography_md, 10, 10)
advanced_stacking('tractography_md' ,tractography_md, 30, 10)
advanced_stacking('tractography_md' ,tractography_md, 50, 10)



 iter  1
=====Validation=====
correlation : 0.021698417877938074
max correlation index : 45
=====Test=====
age corr : -0.0748


 iter  2
=====Validation=====
correlation : 0.05418583191684544
max correlation index : 12
=====Test=====
age corr : 0.1174


 iter  3
=====Validation=====
correlation : 0.026361558493854763
max correlation index : 6
=====Test=====
age corr : 0.0071


 iter  4
=====Validation=====
correlation : 0.0023888380764316565
max correlation index : 9
=====Test=====
age corr : 0.1216


 iter  5
=====Validation=====
correlation : -0.0011681196771064777
max correlation index : 6
=====Test=====
age corr : -0.0676


 iter  6
=====Validation=====
correlation : -0.016200295210136825
max correlation index : 32
=====Test=====
age corr : -0.0645


 iter  7
=====Validation=====
correlation : 0.011541998395863826
max correlation index : 26
=====Test=====
age corr : -0.0751


 iter  8
=====Validation=====
correlation : -0.023349780278518906
max correlation index : 38
=====Test===

In [110]:
advanced_stacking('tractography_od' ,tractography_od, 10, 10)
advanced_stacking('tractography_od' ,tractography_od, 30, 10)
advanced_stacking('tractography_od' ,tractography_od, 50, 10)



 iter  1
=====Validation=====
correlation : 0.032428329814210496
max correlation index : 32
=====Test=====
age corr : -0.0649


 iter  2
=====Validation=====
correlation : -0.008184938258093239
max correlation index : 15
=====Test=====
age corr : 0.1241


 iter  3
=====Validation=====
correlation : 0.016306302557288183
max correlation index : 48
=====Test=====
age corr : 0.1016


 iter  4
=====Validation=====
correlation : 0.01808699184623911
max correlation index : 12
=====Test=====
age corr : 0.0217


 iter  5
=====Validation=====
correlation : 0.009459876899542875
max correlation index : 30
=====Test=====
age corr : -0.1752


 iter  6
=====Validation=====
correlation : -0.0471296905581883
max correlation index : 23
=====Test=====
age corr : 0.0244


 iter  7
=====Validation=====
correlation : -0.02401137745287969
max correlation index : 46
=====Test=====
age corr : 0.1741


 iter  8
=====Validation=====
correlation : -0.019830444364749924
max correlation index : 23
=====Test=====


In [111]:
advanced_stacking('tractography_rd' ,tractography_rd, 10, 10)
advanced_stacking('tractography_rd' ,tractography_rd, 30, 10)
advanced_stacking('tractography_rd' ,tractography_rd, 50, 10)



 iter  1
=====Validation=====
correlation : 0.007960023463753328
max correlation index : 30
=====Test=====
age corr : 0.0495


 iter  2
=====Validation=====
correlation : 0.003952597768319939
max correlation index : 33
=====Test=====
age corr : -0.1430


 iter  3
=====Validation=====
correlation : 0.0017566517619140535
max correlation index : 35
=====Test=====
age corr : -0.0364


 iter  4
=====Validation=====
correlation : 0.0001988598963197648
max correlation index : 32
=====Test=====
age corr : 0.0740


 iter  5
=====Validation=====
correlation : 0.006170301971175366
max correlation index : 23
=====Test=====
age corr : -0.2078


 iter  6
=====Validation=====
correlation : -0.04439691003571443
max correlation index : 37
=====Test=====
age corr : -0.0846


 iter  7
=====Validation=====
correlation : 0.02437381181389203
max correlation index : 45
=====Test=====
age corr : -0.0618


 iter  8
=====Validation=====
correlation : -0.003625333611182747
max correlation index : 56
=====Test=

In [112]:
advanced_stacking('tractography_fss' ,tractography_fss, 10, 10)
advanced_stacking('tractography_fss' ,tractography_fss, 30, 10)
advanced_stacking('tractography_fss' ,tractography_fss, 50, 10)



 iter  1
=====Validation=====
correlation : 0.005004406846970881
max correlation index : 19
=====Test=====
age corr : -0.0955


 iter  2
=====Validation=====
correlation : 0.029250644889232392
max correlation index : 26
=====Test=====
age corr : -0.1026


 iter  3
=====Validation=====
correlation : 0.04072719076797041
max correlation index : 16
=====Test=====
age corr : -0.0276


 iter  4
=====Validation=====
correlation : -0.03894315366333586
max correlation index : 25
=====Test=====
age corr : -0.0725


 iter  5
=====Validation=====
correlation : -0.003563847089979624
max correlation index : 9
=====Test=====
age corr : 0.1059


 iter  6
=====Validation=====
correlation : -0.04871486545744719
max correlation index : 19
=====Test=====
age corr : 0.0074


 iter  7
=====Validation=====
correlation : -0.0021855968061072306
max correlation index : 39
=====Test=====
age corr : -0.0510


 iter  8
=====Validation=====
correlation : 0.02227199828965076
max correlation index : 0
=====Test====

In [113]:
advanced_stacking('tractography_fssl' ,tractography_fssl, 10, 10)
advanced_stacking('tractography_fssl' ,tractography_fssl, 30, 10)
advanced_stacking('tractography_fssl' ,tractography_fssl, 50, 10)



 iter  1
=====Validation=====
correlation : 0.023981118727675155
max correlation index : 49
=====Test=====
age corr : 0.1588


 iter  2
=====Validation=====
correlation : 0.025365963319299346
max correlation index : 25
=====Test=====
age corr : -0.0351


 iter  3
=====Validation=====
correlation : 0.026463192515283677
max correlation index : 10
=====Test=====
age corr : 0.0514


 iter  4
=====Validation=====
correlation : 0.0059611209799927565
max correlation index : 9
=====Test=====
age corr : -0.0363


 iter  5
=====Validation=====
correlation : nan
max correlation index : 14
=====Test=====
age corr : -0.0203


 iter  6
=====Validation=====
correlation : -0.004114207356160334
max correlation index : 30
=====Test=====
age corr : -0.1566


 iter  7
=====Validation=====
correlation : 0.007089618141951117
max correlation index : 14
=====Test=====
age corr : 0.0922


 iter  8
=====Validation=====
correlation : 0.013702515036649559
max correlation index : 41
=====Test=====
age corr : 0.0

**fMRI**

In [114]:
advanced_stacking('f_gamb' ,f_gamb, 10, 10)
advanced_stacking('f_gamb' ,f_gamb, 30, 10)
advanced_stacking('f_gamb' ,f_gamb, 50, 10)



 iter  1
=====Validation=====
correlation : 0.013250037649401952
max correlation index : 20
=====Test=====
age corr : -0.0918


 iter  2
=====Validation=====
correlation : -0.03529834477970881
max correlation index : 2
=====Test=====
age corr : -0.1245


 iter  3
=====Validation=====
correlation : 0.01691577558272433
max correlation index : 23
=====Test=====
age corr : 0.1362


 iter  4
=====Validation=====
correlation : 0.015360110086943598
max correlation index : 22
=====Test=====
age corr : -0.0731


 iter  5
=====Validation=====
correlation : 0.04013743545549081
max correlation index : 0
=====Test=====
age corr : -0.0859


 iter  6
=====Validation=====
correlation : -0.03426787410160977
max correlation index : 29
=====Test=====
age corr : -0.0535


 iter  7
=====Validation=====
correlation : 0.01695310922566494
max correlation index : 41
=====Test=====
age corr : 0.1386


 iter  8
=====Validation=====
correlation : -0.005680366418116045
max correlation index : 42
=====Test=====
a

In [115]:
advanced_stacking('f_lang' ,f_lang, 10, 10)
advanced_stacking('f_lang' ,f_lang, 30, 10)
advanced_stacking('f_lang' ,f_lang, 50, 10)



 iter  1
=====Validation=====
correlation : 0.00984189557803762
max correlation index : 30
=====Test=====
age corr : -0.0754


 iter  2
=====Validation=====
correlation : -0.00018660985832518015
max correlation index : 39
=====Test=====
age corr : -0.0887


 iter  3
=====Validation=====
correlation : -0.002362118854627962
max correlation index : 57
=====Test=====
age corr : 0.1225


 iter  4
=====Validation=====
correlation : 0.0045003176777553494
max correlation index : 1
=====Test=====
age corr : 0.0703


 iter  5
=====Validation=====
correlation : -0.022000079373505965
max correlation index : 38
=====Test=====
age corr : -0.0125


 iter  6
=====Validation=====
correlation : 0.0538522985633716
max correlation index : 38
=====Test=====
age corr : -0.0794


 iter  7
=====Validation=====
correlation : -0.014177600269500728
max correlation index : 18
=====Test=====
age corr : 0.0125


 iter  8
=====Validation=====
correlation : -0.008606400521352852
max correlation index : 43
=====Test

In [116]:
advanced_stacking('f_motor' ,f_motor, 10, 10)
advanced_stacking('f_motor' ,f_motor, 30, 10)
advanced_stacking('f_motor' ,f_motor, 50, 10)



 iter  1
=====Validation=====
correlation : 0.03442686807363231
max correlation index : 38
=====Test=====
age corr : -0.0365


 iter  2
=====Validation=====
correlation : -0.04126764304021761
max correlation index : 35
=====Test=====
age corr : -0.0739


 iter  3
=====Validation=====
correlation : -0.007838039000756167
max correlation index : 31
=====Test=====
age corr : -0.0266


 iter  4
=====Validation=====
correlation : 0.026292018458718636
max correlation index : 29
=====Test=====
age corr : 0.0150


 iter  5
=====Validation=====
correlation : -0.011075021827783891
max correlation index : 17
=====Test=====
age corr : 0.1392


 iter  6
=====Validation=====
correlation : 0.022988595593903254
max correlation index : 43
=====Test=====
age corr : 0.1950


 iter  7
=====Validation=====
correlation : 0.037232861117951536
max correlation index : 13
=====Test=====
age corr : 0.0437


 iter  8
=====Validation=====
correlation : 0.008339844115279474
max correlation index : 9
=====Test=====

In [117]:
advanced_stacking('f_rs' ,f_rs, 10, 10)
advanced_stacking('f_rs' ,f_rs, 30, 10)
advanced_stacking('f_rs' ,f_rs, 50, 10)



 iter  1
=====Validation=====
correlation : -0.0066136336503234945
max correlation index : 39
=====Test=====
age corr : -0.0122


 iter  2
=====Validation=====
correlation : -0.017130572297764657
max correlation index : 7
=====Test=====
age corr : -0.0356


 iter  3
=====Validation=====
correlation : 0.03885082799693813
max correlation index : 46
=====Test=====
age corr : -0.0448


 iter  4
=====Validation=====
correlation : 0.0062872502275075435
max correlation index : 24
=====Test=====
age corr : -0.1341


 iter  5
=====Validation=====
correlation : 0.01882894867540423
max correlation index : 17
=====Test=====
age corr : -0.0505


 iter  6
=====Validation=====
correlation : 0.04103701859902522
max correlation index : 22
=====Test=====
age corr : 0.0915


 iter  7
=====Validation=====
correlation : 0.006628648495484013
max correlation index : 5
=====Test=====
age corr : -0.0506


 iter  8
=====Validation=====
correlation : 0.008494845002849558
max correlation index : 15
=====Test===

In [118]:
advanced_stacking('f_social' ,f_social, 10, 10)
advanced_stacking('f_social' ,f_social, 30, 10)
advanced_stacking('f_social' ,f_social, 50, 10)



 iter  1
=====Validation=====
correlation : 0.016295816664278286
max correlation index : 17
=====Test=====
age corr : -0.0728


 iter  2
=====Validation=====
correlation : -0.014537082227787114
max correlation index : 24
=====Test=====
age corr : -0.0620


 iter  3
=====Validation=====
correlation : 0.024278568932213276
max correlation index : 30
=====Test=====
age corr : -0.0347


 iter  4
=====Validation=====
correlation : 0.013973370865828762
max correlation index : 16
=====Test=====
age corr : -0.0099


 iter  5
=====Validation=====
correlation : 0.03498564144839077
max correlation index : 35
=====Test=====
age corr : 0.0343


 iter  6
=====Validation=====
correlation : -0.005026317614387549
max correlation index : 26
=====Test=====
age corr : 0.0206


 iter  7
=====Validation=====
correlation : 0.0019311138731667268
max correlation index : 40
=====Test=====
age corr : -0.0265


 iter  8
=====Validation=====
correlation : -0.008145127509853474
max correlation index : 5
=====Test=

In [119]:
advanced_stacking('f_wm' ,f_wm, 10, 10)
advanced_stacking('f_wm' ,f_wm, 30, 10)
advanced_stacking('f_wm' ,f_wm, 50, 10)



 iter  1
=====Validation=====
correlation : 0.008601665844418129
max correlation index : 38
=====Test=====
age corr : -0.0110


 iter  2
=====Validation=====
correlation : -0.004331887403262677
max correlation index : 49
=====Test=====
age corr : -0.0199


 iter  3
=====Validation=====
correlation : 0.023928592511815793
max correlation index : 0
=====Test=====
age corr : -0.0738


 iter  4
=====Validation=====
correlation : -0.013117292764009003
max correlation index : 57
=====Test=====
age corr : -0.0857


 iter  5
=====Validation=====
correlation : -0.034196853971707714
max correlation index : 7
=====Test=====
age corr : -0.0216


 iter  6
=====Validation=====
correlation : 0.0328899830032222
max correlation index : 34
=====Test=====
age corr : -0.1334


 iter  7
=====Validation=====
correlation : 0.006451452449968505
max correlation index : 53
=====Test=====
age corr : 0.0828


 iter  8
=====Validation=====
correlation : -0.03527370958839214
max correlation index : 22
=====Test===